In this notebook I will create a training table equivalent to ml_model_run_details

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [4]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

## Transform data v1

In [5]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],
            "Data_selection": row['Data_selection'],  # Data selection
            "Application_date": row['application_date'],
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


# transform_datav2

In [6]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_datav2(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v2",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
            "Data_selection": row['Data_selection'],  # Data selection
            "Application_date": row['application_date'],
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

# SIL V1

## SIL

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [ ]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
    case when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime))
         between '2024-06-01' and '2024-09-30' then 'Dev_Train'
         when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) < '2024-06-01' then 'Pre_Train'
                  else 'Dev_Test' end as Data_selection 
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where cic_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-03-24'
;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a9a896f0-ecd1-493c-aa5f-527eb3eaaa64 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204452, 15)


In [10]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [11]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3148366,1a2ba8e3-0abe-4de8-a3e3-6817a7758038,5d2e6210-86f7-4a2a-831a-2e7f3911b724,0.153677,2026-01-13T15:17:09.526813,2026-01-13T15:17:09.526813,Alpha - CIC-SIL-Model,v1,"{""cic_zero_non_granted_ever_flag"": 1, ""cic_zer...",sil_march 25 models,80bb646a-d254-48a2-983e-efe95e7452ec,2026-01-13T15:17:09.526813,{},,android,Dev_Test,2024-12-28 16:18:53
1,3030231,69c91520-50ed-4d8d-8d57-846e1fff9010,e1ae93c6-24af-4d2d-9b55-512defff7cfa,0.052746,2026-01-13T15:17:09.526813,2026-01-13T15:17:09.526813,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 533.0, ""cic_cn...",sil_march 25 models,7f9be72b-a7c6-48ec-bdd9-e61ed2469e78,2026-01-13T15:17:09.526813,{},,android,Dev_Test,2024-11-16 17:27:47
2,2513224,b5876d99-0dce-4879-80e8-c9359fd5341d,870c04fa-7ff5-4d1a-8363-834a865842bf,0.098351,2026-01-13T15:17:09.526813,2026-01-13T15:17:09.526813,Alpha - CIC-SIL-Model,v1,"{""cic_Personal_Loans_granted_contracts_amt_24M...",sil_march 25 models,bb032850-6a2e-45da-828e-e37e1bacc3c0,2026-01-13T15:17:09.526813,{},,android,Pre_Train,2024-05-13 16:10:11
3,3196219,778a3fed-1007-4daa-afbe-5b74e7723236,ee16af65-e087-4ef2-9657-0ce3ee25c422,0.094580,2026-01-13T15:17:09.527352,2026-01-13T15:17:09.527352,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 367.0, ""cic_ma...",sil_march 25 models,a5989987-7dbc-4ed2-9709-b170ac2ff6a1,2026-01-13T15:17:09.527352,{},,android,Dev_Test,2025-01-17 15:22:53
4,2833630,154417aa-b6fb-4397-a5ea-77be3f00b62f,0c898ab3-ee30-4b7e-ad27-da1abb0c0cfa,0.199273,2026-01-13T15:17:09.527352,2026-01-13T15:17:09.527352,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_...",sil_march 25 models,98fcd66b-ab0b-4986-9867-af6d1c2daf54,2026-01-13T15:17:09.527352,{},,android,Dev_Train,2024-09-09 17:30:54


In [12]:

result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,106370,2024-09-30 08:31:30,2025-03-08 20:55:20
1,Dev_Train,61488,2024-06-01 09:08:21,2024-09-29 20:52:21
2,Pre_Train,36594,2023-01-10 10:33:08,2024-05-31 20:37:27


In [13]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8e831c75-3ff8-47a4-8eab-1d477ca237ba>

##### Alpha Sil Stack Model 

In [ ]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime))
        between '2024-06-01' and '2024-09-30' then 'Dev_Train'
        when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) < '2024-06-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-03-24'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 7c1260ba-7d73-429b-a6b8-4bce05e6b109 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204452, 10)


In [15]:
data.columns
data.rename(columns={'beta_demo_score':'sb_demo_score', 'cic_score':'s_cic_score', 
                    'apps_score':'s_apps_score', 'credo_gen_score':'s_credo_score'}, inplace = True)
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'alpha_stack_score',
       'sb_demo_score', 's_cic_score', 's_apps_score', 's_credo_score',
       'osType', 'application_date', 'Data_selection'],
      dtype='object')

In [16]:
feature_column = ['sb_demo_score',
       's_cic_score', 's_apps_score',
       's_credo_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3230350,def6f8fc-f99a-4caa-921c-46b35c7d8c2d,7845c05e-3bd6-4b84-b23c-1883983066c5,0.121009,2026-01-13T15:18:58.003287,2026-01-13T15:18:58.003287,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.06443327569314217, ""s_cic_...",sil_march 25 models,fcf0a7ce-88df-4371-8be3-9d39b765ffcd,2026-01-13T15:18:58.003287,{},,android,Dev_Test,2025-02-01 16:14:27
1,2970683,88a8a951-90b5-4df1-a0d0-411d5bdf6c7e,ca065690-677a-42f1-8910-5257607f9745,0.116622,2026-01-13T15:18:58.003287,2026-01-13T15:18:58.003287,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.10338891945106625, ""s_cic_...",sil_march 25 models,e0d76d1b-a4bc-4508-838f-4df68ef90c3e,2026-01-13T15:18:58.003287,{},,ios,Dev_Test,2024-10-25 11:09:34
2,2936916,1691ae03-8218-4882-baa7-255a9aa90c91,1b10ec73-3819-4e70-b362-317a40a9ef97,0.270248,2026-01-13T15:18:58.003287,2026-01-13T15:18:58.003287,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.06508418602684608, ""s_cic_...",sil_march 25 models,1253f49c-dd2d-4d5d-821e-4b6c10d74de1,2026-01-13T15:18:58.003287,{},,android,Dev_Test,2024-10-17 15:09:11
3,3098590,1f59722a-a88d-4f68-ac10-2c358b9c81dc,d487b3d4-e406-465e-bdc2-90fb324d5b2d,0.098497,2026-01-13T15:18:58.003287,2026-01-13T15:18:58.003287,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.0726597957450856, ""s_cic_s...",sil_march 25 models,25059e63-4bf2-4cd7-882b-b39714ca57aa,2026-01-13T15:18:58.003287,{},,android,Dev_Test,2024-12-12 11:34:56
4,2908731,de3a6de7-6b70-4771-8c77-0c8fa9425556,444bb59b-eafd-4e76-98de-d3b78c15e37e,0.111751,2026-01-13T15:18:58.003839,2026-01-13T15:18:58.003839,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.037324993601611955, ""s_cic...",sil_march 25 models,4c9193f9-8833-43b5-8c1a-192ed674a084,2026-01-13T15:18:58.003839,{},,android,Dev_Test,2024-10-04 10:33:51


In [17]:

result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,106370,2024-09-30 08:31:30,2025-03-08 20:55:20
1,Dev_Train,61488,2024-06-01 09:08:21,2024-09-29 20:52:21
2,Pre_Train,36594,2023-01-10 10:33:08,2024-05-31 20:37:27


In [18]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8bc5f09b-1a9e-405d-b165-2b13a3b51ed7>

##### Beta Sil App Score

In [ ]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
    case when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime))
         between '2023-12-01' and '2024-06-30' then 'Dev_Train'
         when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) < '2023-12-01' then 'Pre_Train'
                  else 'Dev_Test' end as Data_selection 
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-03-20'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID acdc48db-535e-492b-9153-301c22928948 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (317384, 22)


In [20]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2744528,35c9ebe2-95ae-4d18-b5b9-a7325d06625f,eab14aa1-0a1e-4f2e-8a9f-185aec4d0c4e,0.634178,2026-01-13T15:20:46.743829,2026-01-13T15:20:46.743829,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 27.0, ""app_cn...",sil_march 25 models,49cf87a9-c22a-42c0-b092-0d35c39ae07a,2026-01-13T15:20:46.743829,{},,android,Dev_Test,2024-08-11 18:16:40
1,2860565,ec271fbf-6b77-4382-b1e5-d02e45838eec,71ba186d-ed5f-42ba-8155-0ea600e0b9a3,0.745900,2026-01-13T15:20:46.743829,2026-01-13T15:20:46.743829,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 24.0, ""app_cn...",sil_march 25 models,c2aa7092-c09c-4713-9a26-bd136fc188c5,2026-01-13T15:20:46.743829,{},,android,Dev_Test,2024-09-19 14:30:25
2,2670090,211b2e6f-e903-4df3-8643-cd6e55f9f3ca,23c2c4c7-459b-4e22-97cb-e550a400c20f,0.779553,2026-01-13T15:20:46.743829,2026-01-13T15:20:46.743829,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 29.0, ""app_cn...",sil_march 25 models,2b9d571b-2551-4ff5-bd28-63b5e73881a9,2026-01-13T15:20:46.743829,{},,android,Dev_Test,2024-07-18 15:45:01
3,2676025,4fe9a7bd-d172-43a9-b0a5-eef7e5100016,2125d9f3-c5ec-4dd9-861e-bfbc9b7df33d,0.622489,2026-01-13T15:20:46.743829,2026-01-13T15:20:46.743829,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 37.0, ""app_cn...",sil_march 25 models,613db3a3-c6f0-4d11-8722-edbdc8c7e9f2,2026-01-13T15:20:46.743829,{},,android,Dev_Test,2024-07-20 17:44:17
4,3247949,3ce72175-4d62-4417-9f81-e18d56208715,5f44fd04-fbb2-4022-a1a1-2d88b8a182b2,0.646920,2026-01-13T15:20:46.744588,2026-01-13T15:20:46.744588,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 26.0, ""app_cn...",sil_march 25 models,7e062c88-9687-46ec-929b-a7b328073f61,2026-01-13T15:20:46.744588,{},,android,Dev_Test,2025-02-09 14:40:07


In [21]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,201351,2024-06-30 08:26:05,2025-03-08 20:55:20
1,Dev_Train,85456,2023-12-01 09:24:32,2024-06-29 21:14:46
2,Pre_Train,30577,2023-01-02 10:16:04,2023-11-30 20:32:50


In [23]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=60652a11-44f8-4df3-92c8-892cfde0dd70>

##### Beta SIL Demo Score

In [ ]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime))
        between '2023-07-01' and '2024-06-30' then 'Dev_Train'
        when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) < '2023-07-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-03-20'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 007932c2-6409-4da7-b75e-0a9bcb20979a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349206, 23)


In [25]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3122156,09f41f3f-e636-4f46-9576-d2e1205ebb5f,abe2f94f-d595-4ea0-a074-95a5d3cf871f,0.213475,2026-01-13T15:26:35.470796,2026-01-13T15:26:35.470796,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,00ba581a-452d-418e-b6b0-5a5398c6d2fb,2026-01-13T15:26:35.470796,{},,android,Dev_Test,2024-12-20 18:53:14
1,2684946,3de37935-0ed2-43b4-9e22-10cf004c6ff4,0947c4b2-6b33-46f4-9a50-66a56d826130,0.129745,2026-01-13T15:26:35.470796,2026-01-13T15:26:35.470796,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,010c956d-ef5f-46f3-9c70-0482110a9159,2026-01-13T15:26:35.470796,{},,ios,Dev_Test,2024-07-23 18:23:19
2,2845006,4c806bb2-ad4a-4379-a1fc-94191c44856e,b3438b92-a712-42c0-b007-db0d0f99c630,0.092002,2026-01-13T15:26:35.470796,2026-01-13T15:26:35.470796,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,9bbb965e-6bf9-40f2-9845-c8c22322c02e,2026-01-13T15:26:35.470796,{},,android,Dev_Test,2024-09-14 14:32:50
3,3104057,78be3a51-34a5-4366-8c9f-f947c01b1291,ef10710f-2459-4ba4-a4f3-ebd7c1ffbbaa,0.129704,2026-01-13T15:26:35.471792,2026-01-13T15:26:35.471792,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,34db8ba8-37c4-4d2c-a405-216d63cedc9d,2026-01-13T15:26:35.471792,{},,android,Dev_Test,2024-12-14 14:45:28
4,3298655,1bdf270f-c3d7-4df7-a3dd-fd8393f24d2c,1311b446-db18-4191-81cf-b8be22373bb1,0.026656,2026-01-13T15:26:35.471792,2026-01-13T15:26:35.471792,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,0bce76ad-3fa2-496f-b49f-2d118b52ea71,2026-01-13T15:26:35.471792,{},,android,Dev_Test,2025-03-02 17:35:38


In [26]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,220978,2024-06-30 08:26:05,2025-03-08 21:09:43
1,Dev_Train,120564,2023-07-01 10:22:05,2024-06-29 21:14:46
2,Pre_Train,7664,2023-01-02 10:16:04,2023-06-30 19:37:18


In [27]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4b2cb096-9cd0-408d-b5bc-e150e2d779a5>

##### Beta SIL STACK Score Model

In [ ]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime))
        between '2023-07-01' and '2024-06-30' then 'Dev_Train'
        when date(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) < '2023-07-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-03-20'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 18970d57-203c-4d7e-b05e-dade41bb84d8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349206, 9)


In [29]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3024897,2fdf7a89-5540-49a9-abd3-5f9cb23eddc3,6802b9d0-beb7-4182-bd50-7d273ba3c77e,0.030958,2026-01-13T15:32:37.603093,2026-01-13T15:32:37.603093,Beta - StackScoreModel,v1,"{""apps_score"": 0.4578575414148383, ""credo_gen_...",sil_march 25 models,895254f3-8637-493a-b54a-b0ae9496809f,2026-01-13T15:32:37.603093,{},,android,Dev_Test,2024-11-14 16:21:43
1,2845006,4c806bb2-ad4a-4379-a1fc-94191c44856e,965c1114-f22d-4617-8b0f-d56c70c0c3cd,0.219979,2026-01-13T15:32:37.603093,2026-01-13T15:32:37.603093,Beta - StackScoreModel,v1,"{""apps_score"": 0.6408532899718745, ""credo_gen_...",sil_march 25 models,f55a5ab0-732e-4487-83c7-b34fa3afbdf5,2026-01-13T15:32:37.603093,{},,android,Dev_Test,2024-09-14 14:32:50
2,3040308,b7ae9dd0-976d-450e-bcc2-58c4413885e5,32af3905-9e32-4dc4-a1b0-7b5d863d84f7,0.037701,2026-01-13T15:32:37.604108,2026-01-13T15:32:37.604108,Beta - StackScoreModel,v1,"{""apps_score"": 0.4833233108780895, ""credo_gen_...",sil_march 25 models,a3c5f6a3-0509-4b37-9eaa-17ab9efc6e33,2026-01-13T15:32:37.604108,{},,android,Dev_Test,2024-11-20 12:11:54
3,3298655,1bdf270f-c3d7-4df7-a3dd-fd8393f24d2c,59f9ff43-7c76-40f0-924c-2ec5e97de8e5,0.088499,2026-01-13T15:32:37.604108,2026-01-13T15:32:37.604108,Beta - StackScoreModel,v1,"{""apps_score"": 0.6221757898451064, ""credo_gen_...",sil_march 25 models,cf7cd7f1-1c37-49c6-90a6-9db423f8a598,2026-01-13T15:32:37.604108,{},,android,Dev_Test,2025-03-02 17:35:38
4,3217047,47993136-3adf-4699-b960-916855f51bbc,43b4fdc7-e0b5-4018-aaa1-92cddafbb560,0.031332,2026-01-13T15:32:37.604108,2026-01-13T15:32:37.604108,Beta - StackScoreModel,v1,"{""apps_score"": 0.4942873674327595, ""credo_gen_...",sil_march 25 models,d1b152f6-b880-4643-b23f-1f3cdf1a73b6,2026-01-13T15:32:37.604108,{},,android,Dev_Test,2025-01-26 18:06:45


In [30]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,220978,2024-06-30 08:26:05,2025-03-08 21:09:43
1,Dev_Train,120564,2023-07-01 10:22:05,2024-06-29 21:14:46
2,Pre_Train,7664,2023-01-02 10:16:04,2023-06-30 19:37:18


In [31]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ce827af8-5366-44cb-a7b9-db9c65b81b79>

##### Alpha  - IncomeEstimationModel

In [ ]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


In [ ]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta - IncomeEstimationModel

In [ ]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

# Cash V1

## Cash

##### Alpha Cash Stack Model

###### Trench 1

In [32]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
   IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
    case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
         between '2024-10-01' and '2025-02-28' then 'Dev_Train'
         when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                  else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
< '2025-09-24'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID cdaac4e3-8267-4c88-bb41-8cebabb0f758 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (66557, 11)


In [33]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3657067,5fc56859-3c49-444d-9835-403b35819e07,4f007781-d1ad-4f13-9768-7ddf439a453f,0.584321,2026-01-13T15:39:54.302124,2026-01-13T15:39:54.302124,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.33989991666855446, ""apps_scor...",Cash September 25 Models,c849f4fa-3b0e-4520-86b1-67a2639a77a7,2026-01-13T15:39:54.302124,{},Trench 1,Android,Dev_Test,2025-09-01 14:32:56
1,3263088,8d77858a-1074-4c19-8037-b66a8794d09b,ba7b13be-3721-4a30-a468-dafda989f29b,0.375956,2026-01-13T15:39:54.302124,2026-01-13T15:39:54.302124,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4453998880257759, ""apps_score...",Cash September 25 Models,0b572072-418e-41c1-9715-4800ee1b1978,2026-01-13T15:39:54.302124,{},Trench 1,Android,Dev_Train,2025-02-24 13:11:09
2,3450533,5cbeac78-fce5-4e83-a81a-81e24786fb26,839116fd-1ca5-436e-95de-b5fe4ec9c52e,0.081836,2026-01-13T15:39:54.303130,2026-01-13T15:39:54.303130,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4087135298812963, ""apps_score...",Cash September 25 Models,c09a223b-c8ce-4283-975f-3aa50227dbd9,2026-01-13T15:39:54.303130,{},Trench 1,Android,Dev_Test,2025-05-21 22:19:58
3,3302697,3e1bdfbf-996a-4844-ad8e-7214a6822a51,efdeb905-2767-44ea-8ba8-5ed4be24d36a,0.592083,2026-01-13T15:39:54.303130,2026-01-13T15:39:54.303130,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5152408796847795, ""apps_score...",Cash September 25 Models,7dea547b-91d4-4de2-abeb-60547c9f484e,2026-01-13T15:39:54.303130,{},Trench 1,Android,Dev_Test,2025-03-04 19:21:42
4,3030326,272cfa83-01e2-434c-b4f1-35005e6747d2,1858d1f5-a0b6-47fc-84dd-f0356eb5bb27,0.138488,2026-01-13T15:39:54.303130,2026-01-13T15:39:54.303130,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4500107988839839, ""apps_score...",Cash September 25 Models,eb2d2928-d734-4d66-b934-624f239bea41,2026-01-13T15:39:54.303130,{},Trench 1,Android,Dev_Train,2024-11-16 17:58:13


In [34]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,33302,2025-02-28 00:32:29,2025-09-23 23:57:53
1,Dev_Train,33255,2024-10-01 00:02:08,2025-02-27 23:23:01


In [35]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ae535b0d-62c1-4fb2-93c0-0f039bbdaedf>

###### Trench 2

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
    case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
         between '2024-10-01' and '2025-02-28' then 'Dev_Train'
         when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                  else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 1cff4e3f-c7be-4165-a6f7-e6ede758203e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42288, 11)


In [37]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2980091,40a2d7f2-5715-4ce4-bd75-d23a5f1993f6,8eb461a5-799f-4382-906a-7c267e71d424,0.607124,2026-01-13T16:02:36.550215,2026-01-13T16:02:36.550215,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.6002941754862609, ""apps_score...",Cash September 25 Models,b78ed6b7-15f5-4746-9621-1673692e332d,2026-01-13T16:02:36.550215,{},Trench 2,Android,Dev_Test,2025-08-15 20:20:54
1,3411714,b7f28cce-6554-4379-8134-b71b75934166,04dea5b6-e2b0-4a9e-8cd0-9e4250697159,0.273028,2026-01-13T16:02:36.550215,2026-01-13T16:02:36.550215,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.511866149049422, ""apps_score""...",Cash September 25 Models,84dd8f49-e3e6-4121-b13e-ac7c7c2bde0c,2026-01-13T16:02:36.550215,{},Trench 2,Android,Dev_Test,2025-06-09 12:43:14
2,2501161,e0f2af06-2def-4fa5-9645-d2fcde7aa552,969c9bb7-f0c9-4bac-9ae8-42f6082c6226,0.210858,2026-01-13T16:02:36.550215,2026-01-13T16:02:36.550215,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5614067688700592, ""apps_score...",Cash September 25 Models,a43fe083-e50d-4cd9-8f14-6ac2fe650104,2026-01-13T16:02:36.550215,{},Trench 2,Android,Dev_Test,2025-04-15 22:31:20
3,2986958,c2cfca07-a7e6-4c03-bd52-031ac0b20a81,41cf10b7-3d0e-4299-8935-fc2c0f0606e6,0.148714,2026-01-13T16:02:36.551227,2026-01-13T16:02:36.551227,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4399183219098795, ""apps_score...",Cash September 25 Models,b27957ab-b430-4d30-9811-f41e0ca55598,2026-01-13T16:02:36.551227,{},Trench 2,Android,Dev_Test,2025-08-18 17:53:58
4,2823431,32beb654-1e3d-4de8-beab-62e174806e1e,04657b20-c7a2-4d27-a548-fa755ed9166c,0.746736,2026-01-13T16:02:36.551227,2026-01-13T16:02:36.551227,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4568306581622796, ""apps_score...",Cash September 25 Models,293223a7-eacd-4c1f-94d2-55c54c97ea60,2026-01-13T16:02:36.551227,{},Trench 2,Android,Dev_Train,2024-12-16 22:07:05


In [38]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,25040,2025-02-28 00:10:23,2025-09-23 23:58:17
1,Dev_Train,17248,2024-10-01 00:13:19,2025-02-27 23:20:22


In [39]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ffeca67a-b237-41f7-a2f6-be79e8518722>

###### Trench 3

In [40]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
    case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
         between '2024-10-01' and '2025-02-28' then 'Dev_Train'
         when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                  else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
""" 
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 34d18981-816d-430f-a3a1-b174f5b2b90b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (17937, 11)


In [41]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3302682,3da2fdaa-abbc-46c7-bc42-727c3847b08b,f8db3826-49a4-444d-bfd6-6c27ab8cdfe5,0.518153,2026-01-13T16:04:22.765594,2026-01-13T16:04:22.765594,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.3742863785904835, ""apps_score...",Cash September 25 Models,a1b23f9b-1b65-410d-81c5-2274242b4439,2026-01-13T16:04:22.765594,{},Trench 3,Android,Dev_Test,2025-07-16 19:49:22
1,2820910,9c19519e-df86-44aa-a985-ac80d2efdc9d,8a9b2dd2-d060-4876-86bf-fc3223128663,0.527190,2026-01-13T16:04:22.765594,2026-01-13T16:04:22.765594,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5015278675767348, ""apps_score...",Cash September 25 Models,443f411a-1fd3-4bfd-8589-a1e1feaa8868,2026-01-13T16:04:22.765594,{},Trench 3,Android,Dev_Test,2025-07-12 21:06:21
2,3023251,578fb817-5ca1-4411-865f-982e75ae0f92,8c9aebaa-d719-4100-8428-bc282088758a,0.334583,2026-01-13T16:04:22.765594,2026-01-13T16:04:22.765594,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.37300372241724744, ""apps_scor...",Cash September 25 Models,4d57ca5d-01e4-499c-b74f-562bdfa8df4e,2026-01-13T16:04:22.765594,{},Trench 3,Android,Dev_Test,2025-06-13 21:17:16
3,3279030,0d497415-30ab-42c4-96c6-852f21adb12e,7817452f-9301-4314-b4da-71c5b46af6be,0.549206,2026-01-13T16:04:22.765594,2026-01-13T16:04:22.765594,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.28743581767000415, ""apps_scor...",Cash September 25 Models,c713471f-5f9a-44c2-8b2a-d142173676d8,2026-01-13T16:04:22.765594,{},Trench 3,Android,Dev_Test,2025-03-27 01:29:58
4,3109060,745bf1e0-ea16-4302-b4d3-d32d370429a5,34c07511-01c0-4a45-b437-65186760a0d7,0.278499,2026-01-13T16:04:22.765594,2026-01-13T16:04:22.765594,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.46928903627147445, ""apps_scor...",Cash September 25 Models,d687497f-a72c-4675-8ed2-8925ebddea8c,2026-01-13T16:04:22.765594,{},Trench 3,Android,Dev_Test,2025-07-28 21:27:29


In [42]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,11640,2025-02-28 01:16:40,2025-09-23 23:57:31
1,Dev_Train,6297,2024-10-01 00:42:22,2025-02-27 23:06:28


In [43]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f7c44839-e384-4ad2-93af-522cbf009487>

##### Alpha Cash CIC Model

###### Trench 1

In [48]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection  
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")














Job ID 3939fb01-1705-484b-b8f4-72528b473f24 successfully executed: 100%|██████████|














































































Downloading: 100%|██████████|
The shape of the dataframe is:	 (76716, 19)


In [49]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'ca_cic_score',
       'max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [50]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

Query is running:   0%|          |
Query is running:   0%|          |
Query is running:   0%|          |


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3356846,ab50d9ad-a541-45e8-ae12-40b713c40693,3f199054-b312-45a1-8d47-2ff26fe0a01f,0.527646,2026-01-13T16:15:45.116116,2026-01-13T16:15:45.116116,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1829.0, ""ra...",Cash September 25 Models,6d7f7681-059a-40cc-ae00-fb98b7b1ef89,2026-01-13T16:15:45.116116,{},Trench 1,android,Dev_Test,2025-04-05 03:00:49
1,3617008,16ce37b4-eda5-43bd-9829-7945a7af4889,89012c46-2c76-4fa0-b8df-28a463d02662,0.486690,2026-01-13T16:15:45.116116,2026-01-13T16:15:45.116116,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 991.0, ""rat...",Cash September 25 Models,ed2c1599-0268-4133-b4be-e0963d42e149,2026-01-13T16:15:45.116116,{},Trench 1,android,Dev_Test,2025-08-12 20:50:49
2,3425569,38bda36c-7bff-4804-b5cd-d6e6654bfde4,3d753a57-58da-4dcf-9b86-6655fecc9d09,0.538206,2026-01-13T16:15:45.116116,2026-01-13T16:15:45.116116,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 944.0, ""rat...",Cash September 25 Models,a0d2cb27-54f9-428a-8e6c-d92ab22ca2af,2026-01-13T16:15:45.116116,{},Trench 1,ios,Dev_Test,2025-05-08 05:49:45
3,3609492,c390552f-57e5-4b1a-84eb-7b95889d777f,e8f3237b-8421-4685-b85b-f8694bdf194b,0.556247,2026-01-13T16:15:45.117119,2026-01-13T16:15:45.117119,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 365.0, ""rat...",Cash September 25 Models,8f18d3fc-b773-4925-bf88-608bf75de0da,2026-01-13T16:15:45.117119,{},Trench 1,ios,Dev_Test,2025-08-09 14:41:11
4,3636744,017edbf7-3c00-43fc-8707-edde01cccf95,e8d77365-ebc2-4862-a13f-a17220c40299,0.299486,2026-01-13T16:15:45.117119,2026-01-13T16:15:45.117119,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3746.0, ""ra...",Cash September 25 Models,509a1700-2445-474a-8a1b-61cfd2f38bf0,2026-01-13T16:15:45.117119,{},Trench 1,ios,Dev_Test,2025-08-22 15:21:30


In [51]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,39080,2025-01-31 00:12:40,2025-09-23 23:57:53
1,Dev_Train,37636,2024-09-01 00:00:13,2025-01-30 23:46:11


In [52]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=22f3ecfb-e185-4b64-b14c-3719d54239b3>

###### Trench 2

In [53]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection  
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID e6f4f39a-ccde-451b-892e-b2a7548d432f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (47776, 19)


In [54]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3268248,6b3bae57-e2cd-4245-9222-da4de039d284,3bebf611-77c1-43a2-b11c-17444f116f3a,0.553966,2026-01-13T16:23:06.920054,2026-01-13T16:23:06.920054,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 626.0, ""rat...",Cash September 25 Models,54ddda9a-cb95-4660-9628-fffbe839cd9d,2026-01-13T16:23:06.920054,{},Trench 2,android,Dev_Test,2025-06-04 11:07:03
1,2145816,e3fe8abe-afd4-489b-aaed-1f5599b332f3,c5211c51-0445-452b-894c-288f7f6812ae,0.745684,2026-01-13T16:23:06.920054,2026-01-13T16:23:06.920054,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 763.0, ""rat...",Cash September 25 Models,c03904cc-08b4-47b9-ab11-8df0c267d761,2026-01-13T16:23:06.920054,{},Trench 2,ios,Dev_Train,2024-11-19 18:24:08
2,3386943,efde743f-7609-4eb4-8c00-0788d94361c2,f61a4bbb-60e2-4932-b7bd-79edcfe3b6e0,0.697202,2026-01-13T16:23:06.920054,2026-01-13T16:23:06.920054,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1326.0, ""ra...",Cash September 25 Models,15dc99de-1545-4c10-9232-cdefee64e0e4,2026-01-13T16:23:06.920054,{},Trench 2,android,Dev_Test,2025-09-18 16:07:17
3,2461673,35e26d0b-440f-45ec-a313-4a54197291ba,1e2a307b-dcdd-4257-bf8e-706c4cdc91ad,0.741449,2026-01-13T16:23:06.920054,2026-01-13T16:23:06.920054,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 748.0, ""rat...",Cash September 25 Models,78adcb25-81a8-4ab9-ae2f-5a88e56204f6,2026-01-13T16:23:06.920054,{},Trench 2,ios,Dev_Test,2025-05-20 03:22:37
4,1471107,0f0fbf0f-6832-440e-9648-6bbba50d82c3,14936ada-95af-4e11-b7c3-950ea119b473,0.455173,2026-01-13T16:23:06.921055,2026-01-13T16:23:06.921055,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3675.0, ""ra...",Cash September 25 Models,265ac3ee-465f-4e9e-9f7c-79a0ff235da8,2026-01-13T16:23:06.921055,{},Trench 2,android,Dev_Test,2025-06-07 21:08:21


In [55]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,29166,2025-01-31 00:14:01,2025-09-23 23:58:17
1,Dev_Train,18610,2024-09-01 00:08:25,2025-01-30 23:57:46


In [56]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a34f0c36-2c59-4c47-b328-67177fa396a7>

###### Trench 3

In [57]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ab2de410-4ca1-419e-b058-5ca9b74e631d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (18967, 19)


In [58]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,1792561,72dcbfcd-6a48-48ee-9b51-5e3b18b5ca2a,9d2f3bc8-a0eb-4459-9c31-19e0c7a87f1b,0.346956,2026-01-13T17:42:15.053318,2026-01-13T17:42:15.053318,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 4937.0, ""ra...",Cash September 25 Models,a063eeb3-a7d7-4902-b43f-acbe8a33277f,2026-01-13T17:42:15.053318,{},Trench 3,android,Dev_Train,2024-10-27 14:41:34
1,2695798,a55647ff-c247-4557-bb9a-8986c9ba60b7,0fae005f-5ac3-4299-b919-c1eedcc0e295,0.490301,2026-01-13T17:42:15.053318,2026-01-13T17:42:15.053318,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 332.0, ""rat...",Cash September 25 Models,914eb3de-f7d6-4609-9004-f4516bb057cd,2026-01-13T17:42:15.053318,{},Trench 3,android,Dev_Test,2025-09-07 16:40:59
2,2086509,e4859203-9dce-4bd4-b537-c941febfe12e,8ef2826a-8fc2-4847-98a8-90570cd843d2,0.476005,2026-01-13T17:42:15.053318,2026-01-13T17:42:15.053318,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2438.0, ""ra...",Cash September 25 Models,838a0946-2da5-4ea3-94b0-232092d0c3dc,2026-01-13T17:42:15.053318,{},Trench 3,android,Dev_Test,2025-03-21 08:40:19
3,2333952,06935c14-06b7-4423-a217-9d63ccee3fa3,4a53eb70-358a-44f0-a3b9-ff34af7e1eb7,0.413554,2026-01-13T17:42:15.053318,2026-01-13T17:42:15.053318,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 311.0, ""rat...",Cash September 25 Models,eff288ad-d5c3-48e9-b6b7-d0309499d34a,2026-01-13T17:42:15.053318,{},Trench 3,ios,Dev_Train,2024-10-23 05:46:23
4,2013964,2714b385-f8f0-4c8c-934d-054788a85dab,f266d698-110e-43ce-b4a6-bdd2f53ceb24,0.419928,2026-01-13T17:42:15.054321,2026-01-13T17:42:15.054321,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 28.0, ""rati...",Cash September 25 Models,57583e00-75df-48b1-9159-aac97b9f7e67,2026-01-13T17:42:15.054321,{},Trench 3,ios,Dev_Test,2025-02-08 11:41:59


In [59]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,12971,2025-01-31 00:21:42,2025-09-23 23:57:31
1,Dev_Train,5996,2024-09-01 01:18:28,2025-01-30 21:20:17


In [60]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=26bbdc15-f579-4aa5-87c4-2372f4f8dd33>

##### Beta-Cash-Demo-Model

###### Trench 1

In [61]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
and r.c_demo_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 72ddb5a3-b5dc-4276-a6e9-9ff71d1dd970 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (318201, 18)


In [62]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3554061,c431eb2f-5480-40bd-85a3-ba3548cd180a,d6fbd75b-55f0-4a52-ad07-2c604898a80a,0.638949,2026-01-13T17:46:59.558839,2026-01-13T17:46:59.558839,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,304f27e9-e69c-4487-bca9-5a5bb646eb27,2026-01-13T17:46:59.558839,{},Trench 1,ios,Dev_Test,2025-07-13 11:35:21
1,3219382,60cee0e8-572b-4759-bb6f-e01a90cdef3f,a79474e6-6f88-46ea-8363-7e3c192ddc6b,0.431382,2026-01-13T17:46:59.558839,2026-01-13T17:46:59.558839,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,a7cc7109-78c2-4c2c-88b7-c53366fc1e47,2026-01-13T17:46:59.558839,{},Trench 1,android,Dev_Train,2025-01-27 21:41:48
2,3340010,d32d310c-e741-4b89-af43-738bb1cfabc4,8941958c-2289-4aac-ab02-a8a43f859d68,0.379142,2026-01-13T17:46:59.559422,2026-01-13T17:46:59.559422,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,282c89cd-751f-4cdf-a9c3-5edc1f1e9358,2026-01-13T17:46:59.559422,{},Trench 1,ios,Dev_Test,2025-03-25 15:36:26
3,3085218,59d1d64a-8a0b-49f1-a269-f255a2cc7c65,4baa45c1-0d38-438b-b773-d9feb64ad436,0.557882,2026-01-13T17:46:59.559422,2026-01-13T17:46:59.559422,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,eb11b6d7-09f2-498a-ba7f-07ba3ea15cc9,2026-01-13T17:46:59.559422,{},Trench 1,ios,Dev_Train,2024-12-09 09:14:08
4,3469324,930a6014-c86d-4895-8271-77cd80202862,c1df43f0-a22c-4702-9fc5-f8b3e06b5060,0.460131,2026-01-13T17:46:59.559422,2026-01-13T17:46:59.559422,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,c8d8972b-1f8c-4e98-9bb6-af4f42b3301c,2026-01-13T17:46:59.559422,{},Trench 1,ios,Dev_Test,2025-05-31 12:59:45


In [63]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,192533,2025-01-31 00:03:46,2025-09-23 23:57:53
1,Dev_Train,125668,2024-10-01 00:01:28,2025-01-30 23:59:52


In [64]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a95f1340-4b66-4c9e-a330-7fb4d13a7adf>

###### Trench 2

In [65]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
and r.c_demo_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID d6b81d97-5fd7-41ed-8363-bbca4d51b089 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (122498, 18)


In [66]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3439091,4be2a19c-c54d-4414-9dde-8c1e89887995,43054721-b576-4492-aeac-2b3502458f45,0.569961,2026-01-13T18:08:22.557570,2026-01-13T18:08:22.557570,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,1ae4093e-919d-4948-bb9f-ee653a8911a6,2026-01-13T18:08:22.557570,{},Trench 2,android,Dev_Test,2025-09-14 15:47:13
1,2818342,472d28d9-8bc5-488d-97bf-1d5016b4a69e,5a82f5a2-2e6b-44d3-8dc6-fc16cfa2cd5a,0.460131,2026-01-13T18:08:22.557570,2026-01-13T18:08:22.557570,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,f50172dd-90ff-4ea4-ab6d-f12e4f524465,2026-01-13T18:08:22.557570,{},Trench 2,ios,Dev_Train,2024-12-07 08:52:41
2,2230477,8c9c8b69-e04a-4c40-9d19-f7e964faf1e0,bb281c8e-4393-44d1-bd6c-988b00794c9b,0.558379,2026-01-13T18:08:22.557570,2026-01-13T18:08:22.557570,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,aa43049a-340b-4ed1-ab1a-884331629021,2026-01-13T18:08:22.557570,{},Trench 2,android,Dev_Test,2025-03-22 13:05:26
3,1599991,a05777de-3d6f-4cde-98cf-834d47fcf726,d8214845-47c1-4b6a-ac52-ef537ad23693,0.381481,2026-01-13T18:08:22.558571,2026-01-13T18:08:22.558571,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,75d38e30-5275-4713-b4b3-1e1d204377a4,2026-01-13T18:08:22.558571,{},Trench 2,ios,Dev_Test,2025-08-21 15:57:25
4,1140951,5270d22f-9e28-438a-9c63-fb2bc00052e0,54bf7468-c50a-4902-93d4-924b7cf5751c,0.547600,2026-01-13T18:08:22.558571,2026-01-13T18:08:22.558571,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,537c8dcf-bcc0-40d9-8fa4-0d4899468431,2026-01-13T18:08:22.558571,{},Trench 2,android,Dev_Test,2025-08-04 09:11:46


In [67]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,85138,2025-01-31 00:09:44,2025-09-23 23:58:17
1,Dev_Train,37360,2024-10-01 00:02:41,2025-01-30 23:57:46


In [68]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2488e782-6a0f-43c9-aafe-7e981205df70>

###### Trench 3

In [69]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
and r.c_demo_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 74ff3a24-17ed-482f-becf-7afb05058c81 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (41635, 18)


In [70]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2518859,13cc0348-44cb-40f4-981b-fea0a5e1c365,c1974dd9-a3d0-40ba-b24e-55b79052e43a,0.417618,2026-01-13T18:16:15.227964,2026-01-13T18:16:15.227964,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,301c81ab-158f-4c2d-8d75-489d30ad4da4,2026-01-13T18:16:15.227964,{},Trench 3,ios,Dev_Test,2025-06-27 18:02:55
1,1815252,9fc288fe-0a8c-46f1-b6f6-568a8ccff237,564961f7-ebca-4a75-8416-a653406a91e4,0.362610,2026-01-13T18:16:15.227964,2026-01-13T18:16:15.227964,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,e2484a82-581d-40b7-886e-b71f85a887c0,2026-01-13T18:16:15.227964,{},Trench 3,ios,Dev_Train,2025-01-26 17:46:46
2,3284151,e8806b49-8300-4ae4-b084-844b71637757,48107d50-b384-4cb0-8936-74ae5024f6a5,0.487559,2026-01-13T18:16:15.227964,2026-01-13T18:16:15.227964,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,f10d23f8-ccb1-460d-9663-89f9a35dda84,2026-01-13T18:16:15.227964,{},Trench 3,ios,Dev_Test,2025-09-21 11:37:55
3,2800040,17ccc1be-ec8d-4724-b559-9d17d396c171,37bde9ad-8e87-4e7d-85a8-79d6c9f2ec06,0.425843,2026-01-13T18:16:15.228960,2026-01-13T18:16:15.228960,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,c30bce46-ef46-4fc4-b5ae-c1b1f37bff5a,2026-01-13T18:16:15.228960,{},Trench 3,ios,Dev_Test,2025-02-14 14:48:52
4,2442370,2698b4e8-1fc0-4cba-a4c3-c7da83c2cee2,ce6dba0b-55e3-4d68-9c3b-1d95e695c2da,0.432193,2026-01-13T18:16:15.228960,2026-01-13T18:16:15.228960,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,af035b6e-9ed6-496a-9087-1dba8470d7bf,2026-01-13T18:16:15.228960,{},Trench 3,android,Dev_Test,2025-07-22 14:34:10


In [71]:

result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,31046,2025-01-31 00:21:42,2025-09-23 23:57:31
1,Dev_Train,10589,2024-10-01 00:42:22,2025-01-30 23:37:46


In [72]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=135936ef-7a7b-414f-a004-7c475c4518bc>

##### Beta-Cash-Stack-Model

In [73]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
 from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null 
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 80384eda-62af-4f13-b891-b22f9aa4e7af successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (318108, 10)


In [74]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3695475,7b16df9f-b7e6-4b85-b017-9f717f69c9c7,3bd46977-7af7-4d96-8a00-0330b80b6dab,0.511607,2026-01-13T18:41:49.547759,2026-01-13T18:41:49.547759,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4363299268542517, ""apps_score...",Cash September 25 Models,aae25096-c690-4770-8ea1-a7949601da32,2026-01-13T18:41:49.547759,{},Trench 1,android,Dev_Test,2025-09-21 00:11:16
1,2931424,380f61d3-b61f-468d-8c52-fa3ef9ea607c,28a5c6f5-bfc5-4825-bcce-0cfa446db5d1,0.803008,2026-01-13T18:41:49.547759,2026-01-13T18:41:49.547759,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5217940638582476, ""apps_score...",Cash September 25 Models,299a1f7c-158f-42aa-b719-dc2acaa56dcc,2026-01-13T18:41:49.547759,{},Trench 1,android,Dev_Train,2024-10-11 19:01:20
2,3461387,1ac30637-8e8b-4a23-8011-ffee1854708b,90e53e67-a8fb-4fea-9d7b-2a4d7f5913c6,0.416161,2026-01-13T18:41:49.547759,2026-01-13T18:41:49.547759,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.48369950752401747, ""apps_scor...",Cash September 25 Models,2ac7341e-0784-4345-aed2-9a418967910d,2026-01-13T18:41:49.547759,{},Trench 1,android,Dev_Test,2025-05-27 11:26:04
3,3023301,cb21c49d-0315-4cfd-ac8d-6c7c0d4839a3,efddaba3-6265-4255-92f7-2340da8da92e,0.622702,2026-01-13T18:41:49.547759,2026-01-13T18:41:49.547759,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6001479127285608, ""apps_score...",Cash September 25 Models,2bd9999c-d79d-4cf5-9804-81016ccfb4f2,2026-01-13T18:41:49.547759,{},Trench 1,android,Dev_Train,2024-11-14 00:39:13
4,3540315,e4bd6ea6-a6c2-439d-b071-42c71e02f18b,d3764a3b-da88-43d1-abef-b26beafb8295,0.701954,2026-01-13T18:41:49.547759,2026-01-13T18:41:49.547759,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5953777228203773, ""apps_score...",Cash September 25 Models,53612e4c-4e03-4e64-b4ed-42d7bcb51fae,2026-01-13T18:41:49.547759,{},Trench 1,android,Dev_Test,2025-07-06 10:26:46


In [75]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,163726,2025-02-28 00:05:39,2025-09-23 23:57:53
1,Dev_Train,154382,2024-10-01 00:01:28,2025-02-27 23:59:52


In [76]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=aeeddee7-f400-489b-914c-80b5598a2dd6>

###### Trench 2

In [77]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
 from worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 235d2815-8a0c-4b4b-b25d-b1affe7b1844 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (119665, 10)


In [78]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3386829,90c4ed32-a9ab-442a-bd2b-2ec69fc0a801,5d10a7b4-70ce-49e4-97db-1fda4a7b96da,0.813640,2026-01-13T18:48:26.532601,2026-01-13T18:48:26.532601,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4283907338380405, ""apps_score...",Cash September 25 Models,3ca23b16-de43-493f-80b0-8da18131efba,2026-01-13T18:48:26.532601,{},Trench 2,android,Dev_Test,2025-07-24 09:09:03
1,1773980,a2df33fb-84ac-4922-af24-bba9c8482113,2ce42a56-8a94-4eb1-99a1-3d9c6290457a,0.311020,2026-01-13T18:48:26.532601,2026-01-13T18:48:26.532601,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5213441496176229, ""apps_score...",Cash September 25 Models,63844ab7-1328-48e4-903f-d8d4fa169149,2026-01-13T18:48:26.532601,{},Trench 2,android,Dev_Train,2025-02-26 02:44:56
2,2060145,7ce617be-4a52-42c8-93c6-4b391eff8115,a89bb7fc-be0c-4268-b41a-2f951f0bf9aa,0.504032,2026-01-13T18:48:26.532601,2026-01-13T18:48:26.532601,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6675378333040781, ""apps_score...",Cash September 25 Models,13656d65-f2f5-4a5a-8f52-a6e15ce668cc,2026-01-13T18:48:26.532601,{},Trench 2,android,Dev_Train,2025-02-10 08:03:16
3,3139643,5d342672-0ff4-4485-a3e5-9962c3cf99e2,688d85b0-61d7-4e63-bad9-8baaa4364f54,0.452678,2026-01-13T18:48:26.532601,2026-01-13T18:48:26.532601,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.45362217791035286, ""apps_scor...",Cash September 25 Models,3071d6ff-441a-405a-9856-e5b8c22138b3,2026-01-13T18:48:26.532601,{},Trench 2,android,Dev_Test,2025-07-29 03:05:54
4,3270729,ace736c1-a168-47ee-ae17-ac95fba146c9,523db368-9bbe-422b-a110-da796441e937,0.813391,2026-01-13T18:48:26.532601,2026-01-13T18:48:26.532601,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.552097919532174, ""apps_score""...",Cash September 25 Models,33505cde-f259-4bf5-9c3f-b05e4ae88834,2026-01-13T18:48:26.532601,{},Trench 2,android,Dev_Test,2025-06-23 21:20:15


In [79]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,72880,2025-02-28 00:00:49,2025-09-23 23:58:17
1,Dev_Train,46785,2024-10-01 00:02:41,2025-02-27 23:57:27


In [80]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=28525bb3-42ca-4006-b53e-35b967cdd208>

###### Trench 3

In [81]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-10-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-10-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
 from worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null 
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 962ab047-d8e6-4646-bf18-393d3d18950b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (41609, 10)


In [82]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2204390,3b4747fc-66fd-432e-9ca0-35d99d4a4ac9,4bfefa12-c09c-4cd6-af31-841a626c9689,0.667526,2026-01-13T18:50:08.681496,2026-01-13T18:50:08.681496,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4996458552118163, ""apps_score...",Cash September 25 Models,f5792eaa-49d7-4a8c-917d-8e50f35be964,2026-01-13T18:50:08.681496,{},Trench 3,android,Dev_Test,2025-05-05 06:38:19
1,1139307,14aca0b1-045f-4f9d-99b7-bf3b265ef7cd,86308faa-f14f-4910-b8fb-15e081922bd7,0.359614,2026-01-13T18:50:08.681496,2026-01-13T18:50:08.681496,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4299641914623701, ""apps_score...",Cash September 25 Models,a50dac75-0a2e-46cc-a856-e942df5c5479,2026-01-13T18:50:08.681496,{},Trench 3,android,Dev_Test,2025-09-08 19:00:53
2,2725311,e8176611-2886-47be-ab2d-8e2a98109f6e,fc5555dd-1ee5-472e-a3cd-b50f2505bdb0,0.593206,2026-01-13T18:50:08.681496,2026-01-13T18:50:08.681496,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5205160584495085, ""apps_score...",Cash September 25 Models,94c070f2-2ca7-4b68-a303-9d6c6abeb18f,2026-01-13T18:50:08.681496,{},Trench 3,android,Dev_Test,2025-07-05 06:24:23
3,2989478,15c4efea-6bba-49fe-b6cb-508e1712161c,1f057576-3bdf-4ba0-a7db-3968dc8aaede,0.717613,2026-01-13T18:50:08.681496,2026-01-13T18:50:08.681496,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5470531460533199, ""apps_score...",Cash September 25 Models,aa78287a-fa83-4079-b2c9-0be7711a8e32,2026-01-13T18:50:08.681496,{},Trench 3,android,Dev_Train,2025-01-26 23:45:22
4,1690189,f6e4d5db-4752-4e90-b631-29a2d025da60,3d4db340-31a3-430e-b483-f8593d07215d,0.587249,2026-01-13T18:50:08.681496,2026-01-13T18:50:08.681496,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4339333427574405, ""apps_score...",Cash September 25 Models,216b20e2-be23-481a-979d-4c3734abd42a,2026-01-13T18:50:08.681496,{},Trench 3,android,Dev_Test,2025-09-04 17:16:46


In [83]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,27659,2025-02-28 00:22:32,2025-09-23 23:57:31
1,Dev_Train,13950,2024-10-01 00:42:22,2025-02-27 23:50:34


In [84]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6dbea48a-2e49-4c39-b61e-38154a629dc2>

#####  Beta Cash AppScore Model

###### Trench 1

In [ ]:
## query to add the trench category trench 1 and trench 2

# create or replace table risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy as 
# select * from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored;



# ALTER TABLE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy
# ADD COLUMN trench_category STRING;

# MERGE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy AS target
# USING (
#   SELECT 
#     a.digitalLoanAccountId,
#     CASE 
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) <= 30 THEN 'Trench 1'
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) > 30 THEN 'Trench 2'
#       WHEN b.ln_loan_level_user_type = '1_Repeat Applicant' THEN 'Trench 3'
#       ELSE NULL
#     END AS trench_category,
#     ROW_NUMBER() OVER (PARTITION BY a.digitalLoanAccountId ORDER BY b.ln_appln_submit_datetime DESC) AS row_num
#   FROM risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy a
#   LEFT JOIN prj-prod-dataplatform.risk_mart.applied_loans_20210701_20250930_trans b
#     ON a.digitalLoanAccountId = b.digitalLoanAccountId
# ) AS source
# ON target.digitalLoanAccountId = source.digitalLoanAccountId
# WHEN MATCHED AND source.row_num = 1 THEN
#   UPDATE SET target.trench_category = source.trench_category;

In [85]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-13' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-13' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and r.apps_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 208d0460-f207-4838-a987-d7811df6cfc9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (382443, 14)


In [86]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d', 'osType', 'application_date',
       'Data_selection'],
      dtype='object')

In [87]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382443 entries, 0 to 382442
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   customerId            382443 non-null  int64         
 1   digitalLoanAccountId  382443 non-null  object        
 2   crifApplicationId     382443 non-null  object        
 3   prediction            382443 non-null  float64       
 4   start_time            382443 non-null  object        
 5   end_time              382443 non-null  object        
 6   modelDisplayName      382443 non-null  object        
 7   modelVersionId        382443 non-null  object        
 8   calcFeature           382443 non-null  object        
 9   subscription_name     382443 non-null  object        
 10  message_id            382443 non-null  object        
 11  publish_time          382443 non-null  object        
 12  attributes            382443 non-null  object        
 13 

In [88]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,126229,2025-01-31 00:03:46,2025-09-23 23:50:57
1,Dev_Train,140968,2024-08-13 00:03:27,2025-01-30 23:59:52
2,Pre_Train,115246,2024-01-01 01:03:29,2024-08-12 23:59:28


In [89]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dce1b464-2795-48cf-b0f4-46d4f3100369>

###### Trench 2

In [90]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
 case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-13' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-13' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and r.apps_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ebf1cf71-2546-487f-9f65-205cf0eac21c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (121596, 14)


In [91]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121596 entries, 0 to 121595
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   customerId            121596 non-null  int64         
 1   digitalLoanAccountId  121596 non-null  object        
 2   crifApplicationId     121596 non-null  object        
 3   prediction            121596 non-null  float64       
 4   start_time            121596 non-null  object        
 5   end_time              121596 non-null  object        
 6   modelDisplayName      121596 non-null  object        
 7   modelVersionId        121596 non-null  object        
 8   calcFeature           121596 non-null  object        
 9   subscription_name     121596 non-null  object        
 10  message_id            121596 non-null  object        
 11  publish_time          121596 non-null  object        
 12  attributes            121596 non-null  object        
 13 

In [92]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,54974,2025-01-31 00:10:53,2025-09-23 23:58:17
1,Dev_Train,36671,2024-08-13 00:05:27,2025-01-30 23:47:33
2,Pre_Train,29951,2024-01-01 00:18:57,2024-08-12 23:52:45


In [93]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2f90d010-8f89-4ef9-8ea2-d4580b61b638>

###### Trench 3

In [94]:
sq = """   
 select 
  r.customerId customer_id, 
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,
  r.app_cnt_health_and_fitness_ever_binned app_cnt_health_and_fitness_ever,
  r.app_cnt_productivity_ever_binned app_cnt_productivity_ever, 
  r.app_cnt_rated_for_18plus_ever_binned app_cnt_rated_for_18plus_ever,
  r.app_cnt_books_and_reference_ever_binned app_cnt_books_and_reference_ever, 
  r.app_cnt_gaming_180d_binned app_cnt_gaming_180d,
  r.app_cnt_absence_tag_365d_binned app_cnt_absence_tag_365d,
  r.app_last_payday_install_to_apply_days_binned app_last_payday_install_to_apply_days,
  r.app_cnt_absence_tag_365d_binned,
  r.app_cnt_gaming_180d_binned,
  r.app_cnt_productivity_ever_binned,
  r.app_cnt_rated_for_18plus_ever_binned,
  r.app_cnt_health_and_fitness_ever_binned,
  r.app_cnt_books_and_reference_ever_binned,
  r.app_last_payday_install_to_apply_days_binned,
  r.ln_user_type,
  case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
 case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-13' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-13' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from risk_mart.b_score_model_applied_loans_cash_20240101_20250930_app_scored r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.apps_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-09-24'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 0577f67a-8c04-4eff-a109-97010112dead successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (65874, 21)


In [95]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'ln_user_type']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3328925,2e38701e-e801-456b-8851-a7ed2210fc59,b1714d94-fe30-415c-9a5e-2bbe0843660b,0.458030,2026-01-13T19:09:07.812340,2026-01-13T19:09:07.812340,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.4580296386085596, ""a...",Cash September 25 Models,fa163c5f-16e2-4b4f-9c15-b4ee831b786c,2026-01-13T19:09:07.812340,{},Trench 3,android,Dev_Test,2025-08-07 15:26:14
1,3321804,2a3f1459-e2dd-43c3-ac7f-af51d5b69737,320db13e-3e2c-4fc5-9f69-933198870006,0.566542,2026-01-13T19:09:07.813342,2026-01-13T19:09:07.813342,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5665418924468943, ""a...",Cash September 25 Models,ac551899-9a8d-47c6-bc9e-53de141d015e,2026-01-13T19:09:07.813342,{},Trench 3,android,Dev_Test,2025-06-01 16:44:53
2,2662868,3b3f5d57-b0ee-4daf-8b71-c7fcabf315fb,40979213-8009-473a-992f-781cbc03314a,0.547367,2026-01-13T19:09:07.813342,2026-01-13T19:09:07.813342,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5473667503971049, ""a...",Cash September 25 Models,0467de10-584b-453e-9d9e-8afb8558e785,2026-01-13T19:09:07.813342,{},Trench 3,android,Dev_Test,2025-05-15 18:13:51
3,3198691,66a1ba7e-c40b-4f63-9644-30b06fddab68,1c04baf5-9051-4988-a9eb-764d8fc3f1b3,0.487241,2026-01-13T19:09:07.813342,2026-01-13T19:09:07.813342,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.4872413385711756, ""a...",Cash September 25 Models,6d471e2e-f456-4bcd-b34d-4fdaa2b4ab2f,2026-01-13T19:09:07.813342,{},Trench 3,android,Dev_Test,2025-08-27 22:07:24
4,3190809,191f13d0-bcff-4c78-b10d-23d668d75fb7,c7d6956a-5cf9-4de8-a76b-58fa74bf69dc,0.516257,2026-01-13T19:09:07.813342,2026-01-13T19:09:07.813342,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5162568305837857, ""a...",Cash September 25 Models,8f8bf4cd-8e74-4c5a-b601-937e2ed30f82,2026-01-13T19:09:07.813342,{},Trench 3,android,Dev_Test,2025-08-17 11:17:59


In [96]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,43429,2025-01-31 02:24:00,2025-09-23 23:58:01
1,Dev_Train,12254,2024-08-13 00:29:06,2025-01-30 23:48:10
2,Pre_Train,10191,2023-01-01 10:43:33,2024-08-12 23:00:52


In [97]:
dfd.info()
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65874 entries, 0 to 65873
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customerId            65874 non-null  object        
 1   digitalLoanAccountId  65874 non-null  object        
 2   crifApplicationId     65874 non-null  object        
 3   prediction            65874 non-null  float64       
 4   start_time            65874 non-null  object        
 5   end_time              65874 non-null  object        
 6   modelDisplayName      65874 non-null  object        
 7   modelVersionId        65874 non-null  object        
 8   calcFeature           65874 non-null  object        
 9   subscription_name     65874 non-null  object        
 10  message_id            65874 non-null  object        
 11  publish_time          65874 non-null  object        
 12  attributes            65874 non-null  object        
 13  trenchCategory  

In [98]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dd273320-5e49-4ace-80ea-7d8b371c7638>

# SIL V2

*   Sil-Alpha-CIC-SIL-Model: worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930
*   Sil-Alpha-StackingModel: worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3
*   Sil-Beta-AppsScoreModel:
*   New Applicants (T1, T2): risk_mart.applied_sil_trench1_trench2_loan_jan2024_30sep2025_app_scored, risk_mart.applied_sil_new_applicants_loan_oct0125_oct1325_app_scored
*   Repeat Applicants (T3): risk_mart.b_score_model_applied_loans_sil_20240101_20250930_app_scored, risk_mart.applied_sil_repeat_applicants_loan_oct0125_oct1325_app_scored
*   Sil-Beta-DemoScoreModel: worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015
*   Sil-Beta-StackScoreModel: worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3m

##### 'Alpha - CIC-SIL-Model'

##### Trench 1

In [99]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
 IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
 case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 043a62c5-6136-4019-bde0-3bd9d76a4867 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (224718, 22)


In [100]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType', 'application_date',
       'Data_selection'],
      dtype='object')

In [101]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType,application_date,Data_selection
0,3679669,215f7b94-b4bf-4a4d-aee4-bd23cea0f569,0.432625,Missing,2_New Applicant,1,0,NaN,1.0,9.0,1.0,NaN,1.0,3000.0,0.000000,281.0,NaN,1.000000,NaN,android,2025-09-12 14:53:20,Dev_Test
1,3660170,fb0f1940-0c3f-4767-a2fd-7d8b7a3acf1b,0.397271,Missing,2_New Applicant,0,0,96754.0,NaN,7.0,1.0,NaN,1.0,16206.0,0.750019,321.0,1.000000,NaN,NaN,android,2025-09-02 18:53:28,Dev_Test
2,3381504,9a637d14-62be-49a6-9541-9cc38348a90f,0.304129,NH_Hi,2_New Applicant,0,0,92274.0,1.0,70.0,1.0,NaN,24.0,34000.0,0.532825,30.0,0.501502,8.354082,4.502190,android,2025-04-15 17:02:20,Dev_Test
3,3710960,d694ab7a-ef81-46c8-90eb-e5ef88245eeb,0.412742,Missing,2_New Applicant,0,0,30399.0,NaN,7.0,1.0,1.0,3.0,46000.0,0.911687,126.0,0.400480,NaN,NaN,android,2025-09-28 15:56:31,Dev_Test
4,3693238,d46ce078-0909-4c8b-ba64-7cb9e5f19826,0.202633,Di,2_New Applicant,0,0,14899.0,12.0,5.0,1.0,2.0,4.0,5000.0,0.000000,4.0,1.332000,0.880596,1.696186,android,2025-09-19 18:50:06,Dev_Test


In [102]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3679669,215f7b94-b4bf-4a4d-aee4-bd23cea0f569,b7dee854-3c76-4ab2-85f4-31f96d89a335,0.432625,2026-01-13T19:30:41.269385,2026-01-13T19:30:41.269385,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,11fee300-a794-4031-8049-8c938bdb82a1,2026-01-13T19:30:41.269385,{},Trench 1,android,Dev_Test,2025-09-12 14:53:20
1,3660170,fb0f1940-0c3f-4767-a2fd-7d8b7a3acf1b,b4947153-3047-4505-9c70-6bffcb15f317,0.397271,2026-01-13T19:30:41.269385,2026-01-13T19:30:41.269385,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,311111c5-ecb6-4beb-8fcc-0feb3d40a8a3,2026-01-13T19:30:41.269385,{},Trench 1,android,Dev_Test,2025-09-02 18:53:28
2,3381504,9a637d14-62be-49a6-9541-9cc38348a90f,77d8ea09-f8e2-43f0-999d-476cd63319e6,0.304129,2026-01-13T19:30:41.269923,2026-01-13T19:30:41.269923,cic_model_sil,v2,"{""ScoreRange"": ""NH_Hi"", ""ln_loan_level_user_ty...",Cash November 25 Models,d1d1a723-a615-4e2a-9536-8779f85e66c9,2026-01-13T19:30:41.269923,{},Trench 1,android,Dev_Test,2025-04-15 17:02:20
3,3710960,d694ab7a-ef81-46c8-90eb-e5ef88245eeb,bb4c5a38-b9b0-4bfd-9b9b-575b1b368710,0.412742,2026-01-13T19:30:41.269923,2026-01-13T19:30:41.269923,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,0ec79279-665c-4862-8ea5-b00786973a26,2026-01-13T19:30:41.269923,{},Trench 1,android,Dev_Test,2025-09-28 15:56:31
4,3693238,d46ce078-0909-4c8b-ba64-7cb9e5f19826,7a185c76-0fa7-497c-bd6b-d5de8c114e71,0.202633,2026-01-13T19:30:41.269923,2026-01-13T19:30:41.269923,cic_model_sil,v2,"{""ScoreRange"": ""Di"", ""ln_loan_level_user_type""...",Cash November 25 Models,0ce20364-e6e4-4e87-8c01-b51070d04359,2026-01-13T19:30:41.269923,{},Trench 1,android,Dev_Test,2025-09-19 18:50:06


In [103]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,130466,2025-01-31 09:20:32,2025-11-15 10:23:43
1,Dev_Train,94252,2024-09-01 09:06:34,2025-01-30 20:48:01


In [104]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=17b5b289-2f05-4b66-b552-75fdc3072f82>

##### Trench 2

In [105]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
 case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
 ;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 063e9fbe-aa98-49f9-bab7-885ac6139515 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10175, 22)


In [106]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType', 'application_date',
       'Data_selection'],
      dtype='object')

In [107]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType,application_date,Data_selection
0,2977558,a706b4a2-4017-4424-ae56-977a9bdf41ca,0.429044,Ai,2_New Applicant,0,0,NaN,NaN,10.0,1.0,NaN,NaN,NaN,NaN,733.0,2.012048,NaN,NaN,android,2025-04-17 09:25:13,Dev_Test
1,1744178,e54f26f2-7f7e-40da-9395-d6892d265158,0.417577,Missing,2_New Applicant,0,0,11356.0,NaN,46.0,1.0,2.0,2.0,15200.0,0.926538,26.0,1.501502,NaN,2.707739,android,2025-05-01 09:35:09,Dev_Test
2,1537483,6db628c9-794a-44f9-b9c8-454173037e31,0.232573,Fi,2_New Applicant,0,0,362436.0,5.0,9.0,1.0,NaN,19.0,300000.0,0.619607,55.0,1.994012,1.165463,0.161513,android,2025-01-19 16:19:29,Dev_Train
3,3200058,a0355cd1-8a85-4aa9-9088-1fceeee964fc,0.324313,Bi,2_New Applicant,0,0,55744.0,NaN,9.0,1.0,NaN,NaN,12206.0,NaN,111.0,1.332000,NaN,NaN,android,2025-05-02 12:15:21,Dev_Test
4,1970654,5a5e779d-c702-4586-a5e9-083520d30693,0.423289,Missing,2_New Applicant,0,0,846.0,NaN,33.0,1.0,NaN,NaN,3100.0,NaN,31.0,NaN,NaN,1.000000,android,2025-09-26 18:23:35,Dev_Test


In [108]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2977558,a706b4a2-4017-4424-ae56-977a9bdf41ca,101f40b7-f8f4-4f81-a810-5430c0e298d2,0.429044,2026-01-13T19:36:46.185817,2026-01-13T19:36:46.185817,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,e1e327bc-2a14-44eb-8c9c-b95171a40acf,2026-01-13T19:36:46.185817,{},Trench 2,android,Dev_Test,2025-04-17 09:25:13
1,1744178,e54f26f2-7f7e-40da-9395-d6892d265158,d6cca38e-dc28-4d4d-b4af-7df84039e558,0.417577,2026-01-13T19:36:46.185817,2026-01-13T19:36:46.185817,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,2c78f3d0-33ea-4e80-8ad1-3a4ba0feb662,2026-01-13T19:36:46.185817,{},Trench 2,android,Dev_Test,2025-05-01 09:35:09
2,1537483,6db628c9-794a-44f9-b9c8-454173037e31,3cb2150e-cd21-470f-83ed-5daa321d83dc,0.232573,2026-01-13T19:36:46.185817,2026-01-13T19:36:46.185817,cic_model_sil,v2,"{""ScoreRange"": ""Fi"", ""ln_loan_level_user_type""...",Cash November 25 Models,a6242bf3-e3b7-4c77-82ac-c66c180ca093,2026-01-13T19:36:46.185817,{},Trench 2,android,Dev_Train,2025-01-19 16:19:29
3,3200058,a0355cd1-8a85-4aa9-9088-1fceeee964fc,0870d11a-2cad-4ca6-b300-50e0c6553f96,0.324313,2026-01-13T19:36:46.185817,2026-01-13T19:36:46.185817,cic_model_sil,v2,"{""ScoreRange"": ""Bi"", ""ln_loan_level_user_type""...",Cash November 25 Models,c19726d6-1a62-45f4-818b-cd72349924c8,2026-01-13T19:36:46.185817,{},Trench 2,android,Dev_Test,2025-05-02 12:15:21
4,1970654,5a5e779d-c702-4586-a5e9-083520d30693,458b5228-d83e-4780-ba1c-f4659b3b791d,0.423289,2026-01-13T19:36:46.186818,2026-01-13T19:36:46.186818,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,d77460b8-f5d7-471c-bc50-cd53d0df9608,2026-01-13T19:36:46.186818,{},Trench 2,android,Dev_Test,2025-09-26 18:23:35


In [109]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,6961,2025-01-31 10:09:14,2025-09-30 20:00:41
1,Dev_Train,3214,2024-09-01 09:18:07,2025-01-30 19:30:28


In [110]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6e76123d-cda3-418f-bc11-9bdb3db3a9e3>

##### Trench 3

In [111]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
 case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-01-31' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection 
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 8c7b3848-d5c7-4035-8ba6-eff86f8e106c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11515, 22)


In [112]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType', 'application_date',
       'Data_selection'],
      dtype='object')

In [113]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType,application_date,Data_selection
0,2889605,2d1f1a51-3d6f-4199-a72a-22393639a024,0.163997,Ei,1_Repeat Applicant,0,0,NaN,3.0,0.0,0.0,1.0,2.0,80000.0,0.732603,71.0,2.012048,0.849083,0.740741,android,2024-10-05 13:55:25,Dev_Train
1,3014787,3ecb9870-b2b4-4450-9d4e-8cfce3a863c7,0.055175,Di,1_Repeat Applicant,0,0,7805.0,7.0,0.0,0.0,NaN,1.0,5312.0,0.654400,13.0,NaN,1.262484,3.216216,ios,2025-03-23 17:14:48,Dev_Test
2,2459668,2316a6a3-b03d-4f76-81dd-4303cae9f22b,0.083972,Ai,1_Repeat Applicant,0,0,29481.0,NaN,0.0,0.0,NaN,NaN,10082.0,NaN,18.0,NaN,NaN,0.846491,android,2024-12-26 18:37:58,Dev_Train
3,3166073,73bb53d8-a000-441c-900f-a6d0f1f97e53,0.110153,NH_Ii,1_Repeat Applicant,0,0,NaN,1.0,0.0,0.0,NaN,NaN,5499.0,NaN,29.0,1.994012,1.000000,1.000000,ios,2025-06-02 14:22:05,Dev_Test
4,1699255,998f4f7f-91e5-4db2-9c75-3cafcf394a2f,0.159917,Missing,1_Repeat Applicant,0,0,44440.0,1.0,1.0,1.0,NaN,8.0,50000.0,0.620534,1095.0,NaN,0.867183,NaN,android,2024-09-29 16:54:14,Dev_Train


In [114]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2889605,2d1f1a51-3d6f-4199-a72a-22393639a024,22775691-acb9-4340-8d38-8bf6e70eb12b,0.163997,2026-01-13T19:40:34.630836,2026-01-13T19:40:34.630836,cic_model_sil,v2,"{""ScoreRange"": ""Ei"", ""ln_loan_level_user_type""...",Cash November 25 Models,b691ef8d-da7a-449a-813a-ea2791daea6d,2026-01-13T19:40:34.630836,{},Trench 3,android,Dev_Train,2024-10-05 13:55:25
1,3014787,3ecb9870-b2b4-4450-9d4e-8cfce3a863c7,d817206c-8fd7-4fc7-82de-fcbbf4d67994,0.055175,2026-01-13T19:40:34.631833,2026-01-13T19:40:34.631833,cic_model_sil,v2,"{""ScoreRange"": ""Di"", ""ln_loan_level_user_type""...",Cash November 25 Models,2b9b7292-21b3-4968-8f5a-7c05c1f21049,2026-01-13T19:40:34.631833,{},Trench 3,ios,Dev_Test,2025-03-23 17:14:48
2,2459668,2316a6a3-b03d-4f76-81dd-4303cae9f22b,77bfdc99-bf25-4099-80a5-0c54ce78e73f,0.083972,2026-01-13T19:40:34.631833,2026-01-13T19:40:34.631833,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,7b9edfef-3bbf-4d25-840f-0c50c7ee9e8f,2026-01-13T19:40:34.631833,{},Trench 3,android,Dev_Train,2024-12-26 18:37:58
3,3166073,73bb53d8-a000-441c-900f-a6d0f1f97e53,a3dacfd8-a218-444d-9d0f-26b89a130f0d,0.110153,2026-01-13T19:40:34.631833,2026-01-13T19:40:34.631833,cic_model_sil,v2,"{""ScoreRange"": ""NH_Ii"", ""ln_loan_level_user_ty...",Cash November 25 Models,3666524a-bb44-42df-8add-fe0d8cd44c3e,2026-01-13T19:40:34.631833,{},Trench 3,ios,Dev_Test,2025-06-02 14:22:05
4,1699255,998f4f7f-91e5-4db2-9c75-3cafcf394a2f,91528649-b4b7-49f2-adf3-7d257716b90f,0.159917,2026-01-13T19:40:34.631833,2026-01-13T19:40:34.631833,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,217dfabd-f09c-49ba-857c-4f0a574c4832,2026-01-13T19:40:34.631833,{},Trench 3,android,Dev_Train,2024-09-29 16:54:14


In [115]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,7276,2025-01-31 09:53:58,2025-09-30 19:24:34
1,Dev_Train,4239,2024-09-01 09:57:04,2025-01-30 19:25:52


In [116]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ac934eb4-b3e9-42e5-a612-38d13d1649a0>

##### Alpha Sil Stack Model 

##### Trench 1

In [117]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 12d394b2-bd7b-4c6d-b605-04ee99a3163d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (232107, 11)


In [118]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [119]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type,application_date,Data_selection
0,3455289,0b76ced0-e3e6-4fb1-8a40-adf1dbde75ed,0.334439,0.484401,0.334160,0.045505,0.344859,android,SIL ZERO,2025-05-24 14:59:20,Dev_Test
1,3524965,e55fbb45-f9d0-4bca-b67e-0e2b02082e8d,0.553265,0.403101,0.567253,0.156864,0.354664,android,SIL ZERO,2025-06-28 13:19:03,Dev_Test
2,3316625,e7ba095d-7375-4388-9478-baf63e60fb41,0.626549,0.427249,0.548629,0.111034,0.475238,android,SIL ZERO,2025-03-12 16:30:56,Dev_Test
3,3313554,8fedd43d-8a8d-49ac-b7ca-82d161b3410c,0.172033,0.327076,0.219939,0.066993,0.310822,android,SIL ZERO,2025-03-10 17:48:56,Dev_Test
4,3441499,80525ba0-c521-4078-a94e-816be7fe4f44,0.840483,0.669687,0.586268,0.271209,0.430639,android,SIL ZERO,2025-05-17 11:26:45,Dev_Test


In [120]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3455289,0b76ced0-e3e6-4fb1-8a40-adf1dbde75ed,25e4b0e0-a48f-4c25-9e48-829e7e1bf6d4,0.334439,2026-01-13T19:45:44.300944,2026-01-13T19:45:44.300944,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4844013958355666, ""apps_sc...",Cash November 25 Models,283704d7-5f2f-4475-9aca-dd1165cd07d3,2026-01-13T19:45:44.300944,{},Trench 1,android,Dev_Test,2025-05-24 14:59:20
1,3524965,e55fbb45-f9d0-4bca-b67e-0e2b02082e8d,9386d955-32b5-4329-8353-1c65b77edaab,0.553265,2026-01-13T19:45:44.301942,2026-01-13T19:45:44.301942,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.40310114502296374, ""apps_s...",Cash November 25 Models,6175bebb-5c64-4eaa-bfe3-8fa3700d5bd9,2026-01-13T19:45:44.301942,{},Trench 1,android,Dev_Test,2025-06-28 13:19:03
2,3316625,e7ba095d-7375-4388-9478-baf63e60fb41,5ffb6c57-a52b-43ac-b4c8-ff6060459c43,0.626549,2026-01-13T19:45:44.301942,2026-01-13T19:45:44.301942,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4272489995183345, ""apps_sc...",Cash November 25 Models,7ec48d3f-6d49-4dcb-a211-7f8878ba112d,2026-01-13T19:45:44.301942,{},Trench 1,android,Dev_Test,2025-03-12 16:30:56
3,3313554,8fedd43d-8a8d-49ac-b7ca-82d161b3410c,19082ece-0bf4-4c38-9a55-1a104a2f334b,0.172033,2026-01-13T19:45:44.301942,2026-01-13T19:45:44.301942,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.327075586569325, ""apps_sco...",Cash November 25 Models,9bc4343b-0f6f-42a3-9c77-bdc566b05eb1,2026-01-13T19:45:44.301942,{},Trench 1,android,Dev_Test,2025-03-10 17:48:56
4,3441499,80525ba0-c521-4078-a94e-816be7fe4f44,ad77c276-696c-4ade-ac36-e8cbfccb739e,0.840483,2026-01-13T19:45:44.301942,2026-01-13T19:45:44.301942,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.6696872400742053, ""apps_sc...",Cash November 25 Models,ec1489bc-d374-4f55-a93e-2ce0d1e98b3e,2026-01-13T19:45:44.301942,{},Trench 1,android,Dev_Test,2025-05-17 11:26:45


In [121]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,125194,2025-02-28 08:44:30,2025-11-15 10:23:43
1,Dev_Train,106913,2024-09-01 09:06:34,2025-02-27 20:44:55


In [122]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=27491cca-927d-4e01-9e93-7c3932255ac1>

##### Trench 2

In [123]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 5703fb60-bb4b-49ff-b458-2c7b2a6b341a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10592, 11)


In [124]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [125]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type,application_date,Data_selection
0,3460683,1de8d8ed-4cde-4454-b267-bbac8daf41d6,0.498984,0.493024,0.458013,0.142062,0.288213,android,SIL Competitor,2025-09-07 18:42:08,Dev_Test
1,1564568,2db871b7-e901-41ec-8cf6-43c4f606b062,0.133290,0.255027,0.371919,0.114415,0.060710,android,SIL Competitor,2025-07-20 12:17:53,Dev_Test
2,1584192,ae6f0cdd-f7bc-41b5-a1db-7259de99c129,0.272001,0.471457,0.308813,0.056297,0.240902,android,SIL Competitor,2025-09-02 17:45:59,Dev_Test
3,2247352,8b6090e3-a729-45b4-8b01-51af7049bf7c,0.703342,0.543753,0.599012,0.181607,0.327202,android,SIL Competitor,2025-05-09 16:01:47,Dev_Test
4,2423119,c623f28b-6f5b-45ca-b763-1641379f76ab,0.801030,0.663430,0.490637,0.116854,0.531832,android,SIL Competitor,2025-10-10 16:05:18,Dev_Test


In [126]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3460683,1de8d8ed-4cde-4454-b267-bbac8daf41d6,863c1ca7-920c-4a0c-80dc-a30bcda0696b,0.498984,2026-01-13T19:48:42.831438,2026-01-13T19:48:42.831438,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.49302395045353714, ""apps_s...",Cash November 25 Models,274f3340-6411-488f-9279-2a9de9febf44,2026-01-13T19:48:42.831438,{},Trench 2,android,Dev_Test,2025-09-07 18:42:08
1,1564568,2db871b7-e901-41ec-8cf6-43c4f606b062,2eb39144-58cc-4f16-b7d6-79e225901ee0,0.133290,2026-01-13T19:48:42.832507,2026-01-13T19:48:42.832507,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.2550266731100688, ""apps_sc...",Cash November 25 Models,8502db09-9f79-4f5b-beb2-d539bd1c5cab,2026-01-13T19:48:42.832507,{},Trench 2,android,Dev_Test,2025-07-20 12:17:53
2,1584192,ae6f0cdd-f7bc-41b5-a1db-7259de99c129,c67a0031-2a6a-4d01-bcc1-235ced8433b3,0.272001,2026-01-13T19:48:42.833037,2026-01-13T19:48:42.833037,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.47145685892410155, ""apps_s...",Cash November 25 Models,a0ebe03c-67fa-401a-88fb-a2b6f129f4ac,2026-01-13T19:48:42.833037,{},Trench 2,android,Dev_Test,2025-09-02 17:45:59
3,2247352,8b6090e3-a729-45b4-8b01-51af7049bf7c,c6fb564a-48ca-49e9-aed0-aac9dae76f9c,0.703342,2026-01-13T19:48:42.833037,2026-01-13T19:48:42.833037,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5437529685233264, ""apps_sc...",Cash November 25 Models,834f6fc5-a43b-46e2-b4a6-7d339f6570aa,2026-01-13T19:48:42.833037,{},Trench 2,android,Dev_Test,2025-05-09 16:01:47
4,2423119,c623f28b-6f5b-45ca-b763-1641379f76ab,b652e760-f6b9-413e-9154-04dc99e569e5,0.801030,2026-01-13T19:48:42.833575,2026-01-13T19:48:42.833575,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.6634297227259304, ""apps_sc...",Cash November 25 Models,68d701fd-e431-49be-8a04-32c943de8d08,2026-01-13T19:48:42.833575,{},Trench 2,android,Dev_Test,2025-10-10 16:05:18


In [127]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,6849,2025-02-28 10:00:47,2025-10-13 19:42:37
1,Dev_Train,3743,2024-09-01 09:18:07,2025-02-27 20:07:41


In [128]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3b3c3966-190b-42e2-b4c6-95d4fbd8cb8f>

##### Trench 3

In [129]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-09-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-09-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID ef414746-cf0a-4d9d-9865-4825f5bf6deb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11874, 11)


In [130]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [131]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type,application_date,Data_selection
0,2375360,4c30461a-94ec-4a5a-b7b3-692b93c2bd1a,0.195878,0.189718,0.457167,0.072768,0.152214,android,SIL-Instore,2025-08-15 18:26:02,Dev_Test
1,3128732,cbfc2917-caf1-4bc7-938e-8e3dbd9f8c2c,0.311137,0.386775,0.492697,0.093288,0.116926,android,SIL Competitor,2025-06-25 13:28:48,Dev_Test
2,1850558,a3785991-1e8f-4c4a-88ed-8b670c50406a,0.234351,0.346176,NaN,0.067630,0.074052,ios,SIL Competitor,2025-07-19 17:11:52,Dev_Test
3,1635782,32b92b17-bbf8-4321-937f-7b6b43444414,0.493745,0.331413,0.296944,0.144485,0.509808,android,SIL-Instore,2025-10-01 10:11:29,Dev_Test
4,2558116,6b915d33-bbe3-44ca-9056-97ffd3f50323,0.281340,0.306926,0.374954,0.081440,0.257926,android,SIL Competitor,2025-04-26 09:06:57,Dev_Test


In [132]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2375360,4c30461a-94ec-4a5a-b7b3-692b93c2bd1a,b29bb3a8-fb50-4511-95b9-9987824ede8e,0.195878,2026-01-13T19:49:39.743222,2026-01-13T19:49:39.743222,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.18971761107914087, ""apps_s...",Cash November 25 Models,64667ab4-12d3-428d-9658-2193c0e36732,2026-01-13T19:49:39.743222,{},Trench 3,android,Dev_Test,2025-08-15 18:26:02
1,3128732,cbfc2917-caf1-4bc7-938e-8e3dbd9f8c2c,f003a1ad-aad8-4352-91bb-68f5cf02993c,0.311137,2026-01-13T19:49:39.743222,2026-01-13T19:49:39.743222,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.38677456455732884, ""apps_s...",Cash November 25 Models,c52c062e-8a05-44b7-85c6-846b07676ca4,2026-01-13T19:49:39.743222,{},Trench 3,android,Dev_Test,2025-06-25 13:28:48
2,1850558,a3785991-1e8f-4c4a-88ed-8b670c50406a,76af99fd-c490-411d-a398-27c80871ce3c,0.234351,2026-01-13T19:49:39.743222,2026-01-13T19:49:39.743222,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.34617624394469887, ""s_cred...",Cash November 25 Models,1ed66f8f-5852-4804-ab58-cdc84b9f0a8c,2026-01-13T19:49:39.743222,{},Trench 3,ios,Dev_Test,2025-07-19 17:11:52
3,1635782,32b92b17-bbf8-4321-937f-7b6b43444414,4b07655d-b484-41c7-88f6-10c76f23dd78,0.493745,2026-01-13T19:49:39.743222,2026-01-13T19:49:39.743222,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.33141313319742866, ""apps_s...",Cash November 25 Models,5bfe9581-555d-4329-a021-f268769b9f31,2026-01-13T19:49:39.743222,{},Trench 3,android,Dev_Test,2025-10-01 10:11:29
4,2558116,6b915d33-bbe3-44ca-9056-97ffd3f50323,e56ddf3d-3ae2-4106-a136-ca051df996db,0.281340,2026-01-13T19:49:39.743222,2026-01-13T19:49:39.743222,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.30692614000550067, ""apps_s...",Cash November 25 Models,9ad2fd3a-627e-40d7-a855-a85f55ffb537,2026-01-13T19:49:39.743222,{},Trench 3,android,Dev_Test,2025-04-26 09:06:57


In [133]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,6848,2025-02-28 10:29:08,2025-10-13 18:13:42
1,Dev_Train,5026,2024-09-01 09:57:04,2025-02-27 20:17:03


In [134]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ae772715-a1b2-4cd5-a706-999ef6ebd729>

##### Beta SIL STACK Score Model

##### Trench 1

In [135]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID d6a6ffa9-6953-4ca2-979f-d69deeaecd15 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (343968, 10)


In [136]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [137]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type,application_date,Data_selection
0,3316959,2a7f33cb-d917-4129-87d0-b8d6c4d8be36,0.209399,0.491609,0.202758,0.048612,android,SIL ZERO,2025-03-12 19:32:18,Dev_Test
1,3711094,62f04ad5-5c37-4cdb-a7c4-286f72852bcc,0.288741,0.337625,0.420170,0.090240,android,SIL ZERO,2025-09-28 16:38:24,Dev_Test
2,3631093,5c285898-8d55-4fef-aadc-0a8bdc5e08b6,0.216228,0.382787,0.280757,0.081340,android,SIL ZERO,2025-08-19 17:19:23,Dev_Test
3,3490381,38757ca4-2a54-4ca6-9678-ec734ab4ce4b,0.157428,0.307873,0.258390,0.062376,android,SIL ZERO,2025-06-11 10:10:22,Dev_Test
4,3700846,eb4be3cb-bb92-4497-9660-56d1df4ba040,0.432206,0.353405,0.537222,0.135183,android,SIL ZERO,2025-09-23 15:01:08,Dev_Test


In [138]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3316959,2a7f33cb-d917-4129-87d0-b8d6c4d8be36,bf037e67-3ef1-4aff-a5f8-abefd87bd170,0.209399,2026-01-13T19:51:52.806329,2026-01-13T19:51:52.806329,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.4916089300091016, ""apps_sc...",Cash November 25 Models,7176eda6-9b66-4aad-9425-1547aaf325d9,2026-01-13T19:51:52.806329,{},Trench 1,android,Dev_Test,2025-03-12 19:32:18
1,3711094,62f04ad5-5c37-4cdb-a7c4-286f72852bcc,8eacaacd-9c39-4066-9c8d-2b7ba720c3b0,0.288741,2026-01-13T19:51:52.806329,2026-01-13T19:51:52.806329,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3376254303362985, ""apps_sc...",Cash November 25 Models,fa113800-517d-4d4c-ac9a-bd2123738930,2026-01-13T19:51:52.806329,{},Trench 1,android,Dev_Test,2025-09-28 16:38:24
2,3631093,5c285898-8d55-4fef-aadc-0a8bdc5e08b6,d28351a0-3c89-4d97-966f-32d2254fb8b1,0.216228,2026-01-13T19:51:52.806329,2026-01-13T19:51:52.806329,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.38278721417757106, ""apps_s...",Cash November 25 Models,d17799a5-34b0-4716-bbf2-fa2089a9fcd0,2026-01-13T19:51:52.806329,{},Trench 1,android,Dev_Test,2025-08-19 17:19:23
3,3490381,38757ca4-2a54-4ca6-9678-ec734ab4ce4b,83d4618b-09e2-483f-a6f8-b1d3d64c39f4,0.157428,2026-01-13T19:51:52.806329,2026-01-13T19:51:52.806329,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3078729843590459, ""apps_sc...",Cash November 25 Models,3b894106-f88c-473f-92ef-74a97b08d2f0,2026-01-13T19:51:52.806329,{},Trench 1,android,Dev_Test,2025-06-11 10:10:22
4,3700846,eb4be3cb-bb92-4497-9660-56d1df4ba040,cdef5c4d-f112-4719-8f62-12daf7f1affc,0.432206,2026-01-13T19:51:52.806329,2026-01-13T19:51:52.806329,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3534050185785123, ""apps_sc...",Cash November 25 Models,e65698b6-3fe1-47f4-898f-3f739015f6de,2026-01-13T19:51:52.806329,{},Trench 1,android,Dev_Test,2025-09-23 15:01:08


In [139]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,166098,2025-02-28 08:44:30,2025-11-15 10:23:43
1,Dev_Train,177870,2024-08-01 08:50:39,2025-02-27 20:44:55


In [140]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e23eb7a9-617d-421a-a85a-de6d975635cd>

##### Trench 2

In [141]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID ec00b454-c799-489b-88fe-09862be232a4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13099, 10)


In [142]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [143]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type,application_date,Data_selection
0,2133726,218b32be-e9a7-4a30-ab2c-68d7f191dfad,0.365556,0.280377,0.465822,0.195112,android,SIL Competitor,2025-04-20 19:36:35,Dev_Test
1,2176460,c54ea239-0fee-40ec-9833-0e81cd9d3285,0.553536,0.532772,0.540591,0.101857,android,SIL-Instore,2025-04-19 14:16:59,Dev_Test
2,3544216,ea744a92-6157-45f4-ab07-1322b397a200,0.794362,0.615329,0.611233,0.278965,android,SIL Competitor,2025-08-31 16:53:34,Dev_Test
3,2746162,44e5df42-83cb-46f0-8d9f-7e46d52cbdcf,0.460515,0.576591,0.377210,0.122893,android,SIL-Instore,2025-07-21 10:53:46,Dev_Test
4,1827902,37e57320-b97a-4f5b-87be-4c281e154d2e,0.621612,0.597448,0.560976,0.101235,android,SIL-Instore,2025-03-09 09:40:46,Dev_Test


In [144]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2133726,218b32be-e9a7-4a30-ab2c-68d7f191dfad,5aa85005-032f-4fe1-9180-8bb3a9a21f81,0.365556,2026-01-13T19:55:31.928442,2026-01-13T19:55:31.928442,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.280376520795274, ""apps_sco...",Cash November 25 Models,f056e074-5021-4962-b8b1-73fd36d93b63,2026-01-13T19:55:31.928442,{},Trench 2,android,Dev_Test,2025-04-20 19:36:35
1,2176460,c54ea239-0fee-40ec-9833-0e81cd9d3285,c445847c-d342-4895-9098-5ebade476f06,0.553536,2026-01-13T19:55:31.928442,2026-01-13T19:55:31.928442,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.5327721937584854, ""apps_sc...",Cash November 25 Models,59cff84e-65db-4286-809e-320b264ed799,2026-01-13T19:55:31.928442,{},Trench 2,android,Dev_Test,2025-04-19 14:16:59
2,3544216,ea744a92-6157-45f4-ab07-1322b397a200,8f45ef12-64f7-474e-9acd-9da5778f2bdc,0.794362,2026-01-13T19:55:31.928442,2026-01-13T19:55:31.928442,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.6153294428789763, ""apps_sc...",Cash November 25 Models,118f7c56-1888-4f32-9b64-3ac7b434a1f3,2026-01-13T19:55:31.928442,{},Trench 2,android,Dev_Test,2025-08-31 16:53:34
3,2746162,44e5df42-83cb-46f0-8d9f-7e46d52cbdcf,3e31412e-df86-4e51-a0eb-5e08382b7c5a,0.460515,2026-01-13T19:55:31.928442,2026-01-13T19:55:31.928442,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.576591013920478, ""apps_sco...",Cash November 25 Models,68ae3dd1-d573-49de-8c10-886350772136,2026-01-13T19:55:31.928442,{},Trench 2,android,Dev_Test,2025-07-21 10:53:46
4,1827902,37e57320-b97a-4f5b-87be-4c281e154d2e,bd9626a9-f19a-43ee-8900-eccd1e9d1118,0.621612,2026-01-13T19:55:31.928442,2026-01-13T19:55:31.928442,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.5974477670537301, ""apps_sc...",Cash November 25 Models,48a0ae82-02d3-46d7-95a0-74b3b177b5bd,2026-01-13T19:55:31.928442,{},Trench 2,android,Dev_Test,2025-03-09 09:40:46


In [145]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,7629,2025-02-28 09:17:44,2025-10-13 19:42:37
1,Dev_Train,5470,2024-08-01 09:47:08,2025-02-27 20:07:41


In [146]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=eccf5349-4d90-4fcb-a502-9f1c019ca6d4>

##### Trench 3

In [147]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type,
IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) application_date,
case when IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-02-28' then 'Dev_Train'
        when if(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID ebd4302c-c4cc-4c65-ac24-448f4c62deb2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13802, 10)


In [148]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type', 'application_date', 'Data_selection'],
      dtype='object')

In [149]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type,application_date,Data_selection
0,2717725,b7dff2f1-e7c0-49af-8cc5-0ebcd241dd13,0.429792,0.365774,0.511418,0.142752,android,SIL-Instore,2025-02-28 10:55:27,Dev_Test
1,2997637,4cbe8d18-936e-4caf-b517-45b48e366c0f,0.113558,0.099965,0.344628,0.053957,android,SIL Competitor,2025-05-25 11:16:04,Dev_Test
2,2558997,6ade1585-0c45-480e-a7a2-74df7a50e1ca,0.192594,0.163254,0.445081,0.069591,android,SIL-Instore,2025-05-18 14:04:33,Dev_Test
3,2287801,67771ab5-fef6-462d-a47b-3c799f0de0fb,0.141088,0.145287,0.350653,0.076944,android,SIL ZERO,2025-03-08 17:42:30,Dev_Test
4,3329177,1826b503-e2b8-4d84-967e-e2144e777e47,0.424145,0.278758,0.520506,0.208238,android,SIL Competitor,2025-08-22 18:10:52,Dev_Test


In [150]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2717725,b7dff2f1-e7c0-49af-8cc5-0ebcd241dd13,9653aaa7-5829-4068-a477-fd6b58ca21a4,0.429792,2026-01-13T19:57:06.838972,2026-01-13T19:57:06.838972,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.365774339474942, ""apps_sco...",Cash November 25 Models,523f318d-72f9-4b19-9556-6f1c02c53545,2026-01-13T19:57:06.838972,{},Trench 3,android,Dev_Test,2025-02-28 10:55:27
1,2997637,4cbe8d18-936e-4caf-b517-45b48e366c0f,167738f4-7669-4157-b988-f600bfbabec0,0.113558,2026-01-13T19:57:06.839972,2026-01-13T19:57:06.839972,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.09996512844765226, ""apps_s...",Cash November 25 Models,3253f5eb-666a-4c55-a29e-28bd7ebee09e,2026-01-13T19:57:06.839972,{},Trench 3,android,Dev_Test,2025-05-25 11:16:04
2,2558997,6ade1585-0c45-480e-a7a2-74df7a50e1ca,41d703ef-bc43-4edb-aa5e-565686181414,0.192594,2026-01-13T19:57:06.839972,2026-01-13T19:57:06.839972,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.16325395937206774, ""apps_s...",Cash November 25 Models,6c422a26-2952-41e9-a5a4-b4d34a343264,2026-01-13T19:57:06.839972,{},Trench 3,android,Dev_Test,2025-05-18 14:04:33
3,2287801,67771ab5-fef6-462d-a47b-3c799f0de0fb,0c4a58ec-a6b9-402a-b1b0-33557c21963d,0.141088,2026-01-13T19:57:06.839972,2026-01-13T19:57:06.839972,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.14528749359378812, ""apps_s...",Cash November 25 Models,eff9cabb-0ae4-4544-b51f-20aee82d302f,2026-01-13T19:57:06.839972,{},Trench 3,android,Dev_Test,2025-03-08 17:42:30
4,3329177,1826b503-e2b8-4d84-967e-e2144e777e47,c0a6eddc-d6f1-4f2f-ba05-2c8021951ebc,0.424145,2026-01-13T19:57:06.839972,2026-01-13T19:57:06.839972,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.2787581959549056, ""apps_sc...",Cash November 25 Models,6b7763f1-93d1-4660-bcb3-dc779b51c480,2026-01-13T19:57:06.839972,{},Trench 3,android,Dev_Test,2025-08-22 18:10:52


In [151]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,7404,2025-02-28 10:29:08,2025-10-13 18:13:42
1,Dev_Train,6398,2024-08-01 11:47:48,2025-02-27 20:17:03


In [152]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=978a780b-6dd5-4877-823e-2e2b8f92db02>

##### Beta Sil App Score

##### Trench 1

In [154]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 1'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 4563b786-55d6-4a1d-b154-5d22e560fdc1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (289957, 16)


In [155]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [156]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType,application_date,Data_selection
0,3688576,3e113993-245c-4fdc-8ccc-c8f58d09f4e1,0.187308,2,15.0,2,8,8.0,3.0,1.0,0.0,31.0,0.0,android,2025-09-17 10:49:48,Dev_Test
1,3629378,622cc582-d733-4de0-b1f1-32fdc32c7340,0.384778,6,12.0,2,5,8.0,3.0,2.0,1.0,43.0,1.0,android,2025-08-18 18:52:36,Dev_Test
2,3751680,ef037f6b-330e-4527-bc1d-06303249e164,0.370574,15,26.0,4,11,17.0,11.0,3.0,0.0,91.0,0.0,android,2025-10-18 12:32:48,Dev_Test
3,3427943,6e864211-1396-4abe-966a-e8268e57114c,0.458832,6,12.0,3,4,5.0,10.0,3.0,1.0,28.0,1.0,android,2025-05-09 15:30:42,Dev_Test
4,3597420,0a024573-26fd-4cf9-97d7-66d1b4d07595,0.364491,5,12.0,2,6,9.0,7.0,2.0,0.0,53.0,0.0,android,2025-08-03 19:37:47,Dev_Test


In [157]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

Query is running:   0%|          |


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3688576,3e113993-245c-4fdc-8ccc-c8f58d09f4e1,5490298e-b0db-4c41-8076-d1bd3ce58498,0.187308,2026-01-13T20:20:53.119564,2026-01-13T20:20:53.119564,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 2, ""app_cnt_finance_ev...",Cash November 25 Models,07029af0-2479-42fe-8277-398cc00d39a9,2026-01-13T20:20:53.119564,{},Trench 1,android,Dev_Test,2025-09-17 10:49:48
1,3629378,622cc582-d733-4de0-b1f1-32fdc32c7340,17c664b5-6a8d-457c-bdc7-765f46bae413,0.384778,2026-01-13T20:20:53.119564,2026-01-13T20:20:53.119564,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,e31b2533-bac8-457f-ae52-106f2e029568,2026-01-13T20:20:53.119564,{},Trench 1,android,Dev_Test,2025-08-18 18:52:36
2,3751680,ef037f6b-330e-4527-bc1d-06303249e164,3c801477-cdec-433b-9b1a-8096f442b1b0,0.370574,2026-01-13T20:20:53.121036,2026-01-13T20:20:53.121036,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 15, ""app_cnt_finance_e...",Cash November 25 Models,9cf9674b-384e-40b1-9baf-05b6e8552dc3,2026-01-13T20:20:53.121036,{},Trench 1,android,Dev_Test,2025-10-18 12:32:48
3,3427943,6e864211-1396-4abe-966a-e8268e57114c,3f064dc1-1448-4180-a7ca-0c407a7f7f64,0.458832,2026-01-13T20:20:53.121036,2026-01-13T20:20:53.121036,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,dd165d06-443b-4706-b38e-18e6f88aa793,2026-01-13T20:20:53.121036,{},Trench 1,android,Dev_Test,2025-05-09 15:30:42
4,3597420,0a024573-26fd-4cf9-97d7-66d1b4d07595,63cf90c5-9aec-4263-b1b9-fb130c432c18,0.364491,2026-01-13T20:20:53.121036,2026-01-13T20:20:53.121036,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 5, ""app_cnt_finance_ev...",Cash November 25 Models,2daec2e6-cf70-409c-bd3d-4df8b9a54315,2026-01-13T20:20:53.121036,{},Trench 1,android,Dev_Test,2025-08-03 19:37:47


In [158]:

result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,188335,2025-01-31 08:41:36,2025-11-16 20:59:53
1,Dev_Train,101622,2024-10-01 08:52:22,2025-01-30 20:48:01


In [159]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bf5a5624-ae52-4d12-83d3-accbf7b813d1>

##### Trench 2

In [160]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 2'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 961b46f3-c7ed-4771-9606-a5afcf056981 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11680, 16)


In [161]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [162]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType,application_date,Data_selection
0,3026188,4f058f3a-7e7b-4e40-9d50-0af3f084af8a,0.396767,12,21.0,5,13,13.0,10.0,3.0,0.0,74.0,0.0,android,2025-03-28 15:24:30,Dev_Test
1,1284200,03ea79fc-e6e5-4903-9c79-22fb63f07ca3,0.588936,3,18.0,0,6,18.0,10.0,0.0,0.0,51.0,0.0,android,2025-06-26 14:09:42,Dev_Test
2,3234232,45895863-0ef8-4d6e-bdcf-a0c861176ee9,0.274211,7,20.0,3,7,15.0,14.0,3.0,0.0,83.0,0.0,android,2025-04-14 18:57:57,Dev_Test
3,3184721,1290ba06-6919-4e9c-ba51-ead6c8b1c7f1,0.521990,18,32.0,4,10,27.0,30.0,3.0,6.0,83.0,2.0,android,2025-09-28 17:34:11,Dev_Test
4,1272938,79bff153-8450-40e6-b2a8-d339a11d7796,0.256678,1,22.0,0,1,7.0,9.0,0.0,0.0,63.0,0.0,android,2025-06-11 09:10:28,Dev_Test


In [163]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3026188,4f058f3a-7e7b-4e40-9d50-0af3f084af8a,3b2207c1-6308-44bc-93d3-8d7d5d4713d7,0.396767,2026-01-13T20:22:23.238912,2026-01-13T20:22:23.238912,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 12, ""app_cnt_finance_e...",Cash November 25 Models,b09b20c0-9422-46cd-b375-db0ad444985e,2026-01-13T20:22:23.238912,{},Trench 2,android,Dev_Test,2025-03-28 15:24:30
1,1284200,03ea79fc-e6e5-4903-9c79-22fb63f07ca3,e661b89b-bd30-48fa-87f2-ce9a1db3ccbe,0.588936,2026-01-13T20:22:23.239373,2026-01-13T20:22:23.239373,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 3, ""app_cnt_finance_ev...",Cash November 25 Models,336c0482-dcbf-4db4-b41b-1051d2ec29c8,2026-01-13T20:22:23.239373,{},Trench 2,android,Dev_Test,2025-06-26 14:09:42
2,3234232,45895863-0ef8-4d6e-bdcf-a0c861176ee9,17d280dc-abd8-4f60-a6a9-98832facf391,0.274211,2026-01-13T20:22:23.239373,2026-01-13T20:22:23.239373,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 7, ""app_cnt_finance_ev...",Cash November 25 Models,1a04d864-f355-47d2-9f66-eb33e00dc0f2,2026-01-13T20:22:23.239373,{},Trench 2,android,Dev_Test,2025-04-14 18:57:57
3,3184721,1290ba06-6919-4e9c-ba51-ead6c8b1c7f1,35097de8-9199-4592-b4a2-091c4431255e,0.521990,2026-01-13T20:22:23.239881,2026-01-13T20:22:23.239881,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 18, ""app_cnt_finance_e...",Cash November 25 Models,3409c7bf-f031-4a58-bc64-38df5f6e05de,2026-01-13T20:22:23.239881,{},Trench 2,android,Dev_Test,2025-09-28 17:34:11
4,1272938,79bff153-8450-40e6-b2a8-d339a11d7796,36df358b-dbb9-4b1b-a1fc-da85886daec2,0.256678,2026-01-13T20:22:23.239881,2026-01-13T20:22:23.239881,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 1, ""app_cnt_finance_ev...",Cash November 25 Models,a6976c68-a5f2-445e-a5dc-56308da25e30,2026-01-13T20:22:23.239881,{},Trench 2,android,Dev_Test,2025-06-11 09:10:28


In [164]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,8465,2025-01-31 10:35:47,2025-11-16 19:52:42
1,Dev_Train,3215,2024-10-01 09:55:10,2025-01-30 19:30:28


In [165]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=adb5432a-24b5-4a1b-8203-d5128f8db97a>

##### Trench 3

In [166]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_productivity_ever,
app_cnt_rated_for_3plus_ever, 
app_cnt_books_and_reference_ever,
app_cnt_tools_ever, 
app_median_time_bw_installed_mins_3d,
app_median_time_bw_installed_mins_30d,
app_cnt_communication_ever, 
app_cnt_finance_90d,
app_cnt_absence_tag_180d, 
app_cnt_shopping_ever,
app_cnt_social_ever, 
app_cnt_driver_90d, 
app_cnt_payday_365d,
app_cnt_driver_365d, 
app_cnt_music_and_audio_ever,
app_cnt_finance_180d, 
app_cnt_art_and_design_ever,
app_cnt_gaming_90d, 
app_avg_time_bw_installed_mins_30d,
app_cnt_education_ever,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from risk_mart.applied_sil_repeat_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ddb82e0c-a4cc-4fcb-846f-2ef47627b945 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (19768, 26)


In [167]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',
       'osType', 'application_date', 'Data_selection'],
      dtype='object')

In [168]:
feature_column = ['app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3138748,adb8a8d0-84e8-4ea3-9773-97dd331601f9,3338827e-2bbe-4887-926c-0f1401c87a69,0.559412,2026-01-13T20:23:14.831282,2026-01-13T20:23:14.831282,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 1.0, ""app_cnt_ra...",Cash November 25 Models,76f2b47b-f510-43b1-ae91-98888c0f6a4d,2026-01-13T20:23:14.831282,{},Trench 3,android,Dev_Test,2025-06-28 12:35:17
1,2913664,e790209f-1ea2-49c3-90f5-c2989b9de6e9,c0d960de-5bb6-4cc0-9215-001bcec52776,0.244783,2026-01-13T20:23:14.832278,2026-01-13T20:23:14.832278,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 7.0, ""app_cnt_ra...",Cash November 25 Models,dc7f6bd1-7131-4c9b-942e-6f350da01b92,2026-01-13T20:23:14.832278,{},Trench 3,android,Dev_Test,2025-06-18 15:04:27
2,3216798,3ef0c933-9c1a-4791-ba86-5ea78a9ae226,80816b46-4698-404b-b8e5-8d30aa0ce9a2,0.494901,2026-01-13T20:23:14.832278,2026-01-13T20:23:14.832278,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 2.0, ""app_cnt_ra...",Cash November 25 Models,0d99f680-07f3-47f0-a2e4-1a2ba94344d8,2026-01-13T20:23:14.832278,{},Trench 3,android,Dev_Test,2025-09-13 10:36:44
3,3275779,0651b874-27c3-4c35-b819-6732c8934673,06a99b2e-2f7a-47a8-af15-13cd3fcbb3db,0.466586,2026-01-13T20:23:14.832278,2026-01-13T20:23:14.832278,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 2.0, ""app_cnt_ra...",Cash November 25 Models,875254a1-e935-4908-aaf4-b133733df669,2026-01-13T20:23:14.832278,{},Trench 3,android,Dev_Test,2025-09-12 15:20:42
4,2465198,3912d7be-cd7a-4051-b018-47b03b943f18,95a19a54-50ed-404d-9611-3081cbdb2af5,0.492280,2026-01-13T20:23:14.832278,2026-01-13T20:23:14.832278,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 1.0, ""app_cnt_ra...",Cash November 25 Models,f1108586-9192-428a-ae89-a24495e75437,2026-01-13T20:23:14.832278,{},Trench 3,android,Dev_Train,2024-10-20 18:58:02


In [169]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [170]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19768 entries, 0 to 19767
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customerId            19768 non-null  int64         
 1   digitalLoanAccountId  19768 non-null  object        
 2   crifApplicationId     19768 non-null  object        
 3   prediction            19768 non-null  float64       
 4   start_time            19768 non-null  object        
 5   end_time              19768 non-null  object        
 6   modelDisplayName      19768 non-null  object        
 7   modelVersionId        19768 non-null  object        
 8   calcFeature           19768 non-null  object        
 9   subscription_name     19768 non-null  object        
 10  message_id            19768 non-null  object        
 11  publish_time          19768 non-null  object        
 12  attributes            19768 non-null  object        
 13  trenchCategory  

In [171]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,14612,2025-01-31 09:53:58,2025-11-16 20:02:51
1,Dev_Train,5156,2024-10-01 10:37:42,2025-01-30 19:25:52


In [172]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1b946bc5-468a-4097-8d58-1a46dcbc48a8>

##### Beta SIL Demo Score

##### Trench 1

In [173]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 1'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 739a41e0-2790-4e80-8b9b-e1a4755213ab successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (345905, 21)


In [174]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [175]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3568158,76dd05cc-2406-4ba1-970e-32931371c180,5de2a856-0e97-459c-9f29-87ec77c80d77,0.232384,2026-01-13T20:24:43.671862,2026-01-13T20:24:43.671862,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,d239d8c1-98a5-43ee-97f8-c533ee2c37c0,2026-01-13T20:24:43.671862,{},Trench 1,Android,Dev_Test,2025-07-20 15:03:34
1,2744281,4dd7d3df-ae9d-42e8-8519-52964e4c683d,bf8b565b-8574-4093-8827-7461054862a4,0.290544,2026-01-13T20:24:43.672867,2026-01-13T20:24:43.672867,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,b1e8a77f-408f-4fe5-a06e-c213db6bbfcc,2026-01-13T20:24:43.672867,{},Trench 1,Android,Dev_Train,2024-08-11 17:04:07
2,3408942,eabd14a6-7c83-45af-aba9-901a91d54a30,ad584bc8-9a96-432c-88ba-a907945b41c0,0.262084,2026-01-13T20:24:43.672867,2026-01-13T20:24:43.672867,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,2a7d2f16-cf83-46e6-b2a6-68bb39a7dbf3,2026-01-13T20:24:43.672867,{},Trench 1,Android,Dev_Test,2025-04-29 14:58:07
3,3556808,72f29a6a-2d27-4217-b29b-bcb32cb57791,b5507a26-9124-4896-b41d-07310fc2e2b2,0.414957,2026-01-13T20:24:43.672867,2026-01-13T20:24:43.672867,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,64d10f9e-625c-4866-b08b-4d53101b3408,2026-01-13T20:24:43.672867,{},Trench 1,Android,Dev_Test,2025-07-14 16:20:13
4,3693173,f676a2a3-a9c1-4599-bc96-dd9d640251e6,99b43761-0085-459a-9749-23be54ff2629,0.295337,2026-01-13T20:24:43.672867,2026-01-13T20:24:43.672867,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,4019a771-d323-4c88-ae86-3b31c0187d35,2026-01-13T20:24:43.672867,{},Trench 1,Android,Dev_Test,2025-09-19 17:35:05


In [176]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [177]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345905 entries, 0 to 345904
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   customerId            345905 non-null  int64         
 1   digitalLoanAccountId  345905 non-null  object        
 2   crifApplicationId     345905 non-null  object        
 3   prediction            345905 non-null  float64       
 4   start_time            345905 non-null  object        
 5   end_time              345905 non-null  object        
 6   modelDisplayName      345905 non-null  object        
 7   modelVersionId        345905 non-null  object        
 8   calcFeature           345905 non-null  object        
 9   subscription_name     345905 non-null  object        
 10  message_id            345905 non-null  object        
 11  publish_time          345905 non-null  object        
 12  attributes            345905 non-null  object        
 13 

In [178]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,186542,2025-01-31 08:41:36,2025-11-15 10:23:43
1,Dev_Train,159363,2024-08-01 08:50:39,2025-01-30 20:48:01


In [179]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e00a78e5-f6a2-4183-82f2-36100a2cf695>

##### Trench 2

In [180]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 2'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 58b4c54e-8650-42eb-a389-78c62a7d1b59 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13192, 21)


In [181]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [182]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3252597,69cf484e-7632-4e4e-ac8f-7698d457c672,0764aa0d-5435-4fdd-b6f2-81df2249ed79,0.516306,2026-01-13T20:26:30.689399,2026-01-13T20:26:30.689399,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,1ec4401b-6eb8-483e-84d9-6802bb6e4c15,2026-01-13T20:26:30.689399,{},Trench 2,Android,Dev_Test,2025-05-25 19:20:57
1,2288737,f8710b8e-9ab1-46b0-a612-003d156ce04d,c47dd8f7-24b4-4755-9e59-5649e9a74735,0.414301,2026-01-13T20:26:30.689939,2026-01-13T20:26:30.689939,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,d0484830-8f08-4051-9ced-45406f2219a6,2026-01-13T20:26:30.689939,{},Trench 2,iOS,Dev_Test,2025-09-20 16:35:30
2,3441535,25b84551-a99c-450d-94b4-0274e3c9c9a4,caff2b5c-0912-4add-a681-05a9b6e85634,0.355712,2026-01-13T20:26:30.689939,2026-01-13T20:26:30.689939,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,840c3b39-08b7-4450-a0f1-c0a2a7be9ea6,2026-01-13T20:26:30.689939,{},Trench 2,Android,Dev_Test,2025-08-30 16:31:42
3,3367691,9888b2a0-aace-4ec6-a130-4c41569feb93,1214008f-8ecc-48cf-bb7e-af27126f192c,0.421169,2026-01-13T20:26:30.690474,2026-01-13T20:26:30.690474,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,2a874d82-d619-4e77-b090-5d0aee9654a8,2026-01-13T20:26:30.690474,{},Trench 2,Android,Dev_Test,2025-06-23 12:46:22
4,2190711,723ff05e-f608-4c40-9718-708a3fd53372,84329070-2068-4f26-a361-4f9a3f72d86f,0.603849,2026-01-13T20:26:30.691008,2026-01-13T20:26:30.691008,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,a791c097-e8b8-4c66-8145-fb296f26c7e5,2026-01-13T20:26:30.691008,{},Trench 2,Android,Dev_Test,2025-07-02 11:19:04


In [183]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [184]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13192 entries, 0 to 13191
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customerId            13192 non-null  int64         
 1   digitalLoanAccountId  13192 non-null  object        
 2   crifApplicationId     13192 non-null  object        
 3   prediction            13192 non-null  float64       
 4   start_time            13192 non-null  object        
 5   end_time              13192 non-null  object        
 6   modelDisplayName      13192 non-null  object        
 7   modelVersionId        13192 non-null  object        
 8   calcFeature           13192 non-null  object        
 9   subscription_name     13192 non-null  object        
 10  message_id            13192 non-null  object        
 11  publish_time          13192 non-null  object        
 12  attributes            13192 non-null  object        
 13  trenchCategory  

In [185]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,8432,2025-01-31 10:09:14,2025-10-15 20:32:43
1,Dev_Train,4760,2024-08-01 09:47:08,2025-01-30 19:30:28


In [186]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9eb3e837-8c21-4761-a163-39bb65151b09>

##### Trench 3

In [187]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) application_date,
case when IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)
        between '2024-08-01' and '2025-01-31' then 'Dev_Train'
        when if(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2024-08-01' then 'Pre_Train'
                else 'Dev_Test' end as Data_selection
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 3'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) < '2025-11-17'
 ;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 12960a8d-bf70-4223-88b8-6266d6865e0f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13866, 21)


In [188]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [189]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3158927,672e02b1-5e90-41dd-8cb7-4dcc4dab87ca,3cf96d20-fcf9-4e4c-afaf-fad1c4cb71a4,0.129800,2026-01-13T20:26:41.883743,2026-01-13T20:26:41.883743,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,c2c2487e-3157-41d8-a89a-4cedb6e62587,2026-01-13T20:26:41.883743,{},Trench 3,Android,Dev_Test,2025-07-09 10:33:07
1,2679546,7535cbec-20e2-45f5-a020-533c874ba7c0,faece33e-e8c1-40d8-bfce-5e5a038f8e2d,0.222755,2026-01-13T20:26:41.884754,2026-01-13T20:26:41.884754,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,b7a461a4-4b36-4582-a616-6542951da53b,2026-01-13T20:26:41.884754,{},Trench 3,Android,Dev_Train,2024-08-16 19:40:43
2,3084921,df4f2100-101b-4d51-8a24-c2ca81faa3c5,4e7078bb-334c-40cf-9319-99bb57431611,0.226096,2026-01-13T20:26:41.884754,2026-01-13T20:26:41.884754,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,e6bfcfe9-6d8b-4de1-a5cb-7fff5bf19ec7,2026-01-13T20:26:41.884754,{},Trench 3,Android,Dev_Test,2025-04-26 10:55:32
3,2343791,120b398d-dc7e-4e67-997a-ecc8336a7ba4,1dca409b-7697-41c2-bb17-fa59a6a9d301,0.373451,2026-01-13T20:26:41.884754,2026-01-13T20:26:41.884754,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,166a4307-edf2-4ae1-abb2-cfdd7fa2164f,2026-01-13T20:26:41.884754,{},Trench 3,Android,Dev_Test,2025-08-15 14:13:37
4,2364473,d8c39202-a82e-42d1-af93-c4a8e1dc12bf,dc18b207-fbe9-42a8-ac51-add7c440df5d,0.195290,2026-01-13T20:26:41.884754,2026-01-13T20:26:41.884754,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,e1671740-69b3-4604-b246-f1d6e93ea2a1,2026-01-13T20:26:41.884754,{},Trench 3,Android,Dev_Train,2024-08-10 10:51:13


In [190]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [191]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13866 entries, 0 to 13865
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customerId            13866 non-null  int64         
 1   digitalLoanAccountId  13866 non-null  object        
 2   crifApplicationId     13866 non-null  object        
 3   prediction            13866 non-null  float64       
 4   start_time            13866 non-null  object        
 5   end_time              13866 non-null  object        
 6   modelDisplayName      13866 non-null  object        
 7   modelVersionId        13866 non-null  object        
 8   calcFeature           13866 non-null  object        
 9   subscription_name     13866 non-null  object        
 10  message_id            13866 non-null  object        
 11  publish_time          13866 non-null  object        
 12  attributes            13866 non-null  object        
 13  trenchCategory  

In [192]:

result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,8379,2025-01-31 09:53:58,2025-10-15 19:10:20
1,Dev_Train,5487,2024-08-01 11:47:48,2025-01-30 19:25:52


In [193]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260113"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=33bc2b35-62b4-4483-95a6-06cd347800f8>

# End